# 해당 파일 설명
이 코드를 모두 실행하고 나면, /converted 경로에 다음의 파일이 생성됩니다.

1. dangjin_weather.csv
2. sinpyeong_weather.csv
3. seosan_seather.csv
4. songsan_weather, _INT, _utc, _utc_INT .csv
5. songak_weather, _INT, _utc, _utc_INT .csv

## 추가 설명
위 파일은, 기상 데이터를 시간에 맞게 정제한 데이터 입니다.

위 파일을 기준으로 특정 datetime에 대해 특성을 얻기 위한 interpolation 함수를 dictionary 형태로 뽑아냅니다. 해당 dictionary는 /model 에 ipo_dict.pkl 로 저장되어 있습니다.

# 1. Google drive Mount

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# 2. Define Directory

In [ ]:
# 사용하기 전 변경해주어야 하는 파라미터
BASE_WORKING_DIR = "/content/gdrive/My Drive/weather_bigcontest/4_code/"
# BASE_WORKING_DIR = "/content/gdrive/My Drive/Colab Notebooks/weather_bigcontest/4_code/"
USERNAME = "park"

In [ ]:
import os
# 사용하는 디렉토리 정의
DATA_BASE_PATH = os.path.join(BASE_WORKING_DIR, "data")
DATA_RAW_PATH = os.path.join(DATA_BASE_PATH, "raw")
DATA_EXPORT_PATH = os.path.join(DATA_BASE_PATH, "export")
DATA_CONV_USER_PATH = os.path.join(DATA_BASE_PATH, USERNAME)
DATA_CONV_PATH = os.path.join(DATA_BASE_PATH, "converted")
MODEL_PATH = os.path.join(BASE_WORKING_DIR, "model")
RESULT_PATH = os.path.join(BASE_WORKING_DIR, "result")

WORKING_DIR = os.path.join(BASE_WORKING_DIR, USERNAME)

print('Current Work Directory :',  os.getcwd(), '\n')
print('Username :',  USERNAME, '\n')

print("DATA_BASE_PATH : ", DATA_BASE_PATH)
print("DATA_RAW_PATH : ", DATA_RAW_PATH)
print("DATA_EXPORT_PATH : ", DATA_EXPORT_PATH)
print("DATA_CONV_USER_PATH : ", DATA_CONV_USER_PATH)
print("DATA_CONV_PATH : ", DATA_CONV_PATH)

print("MODEL_PATH : ", MODEL_PATH)

print("RESULT_PATH : ", RESULT_PATH)

print("WORKING_DIR : ", WORKING_DIR)


Current Work Directory : /content 

Username : park 

DATA_BASE_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data
DATA_RAW_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/raw
DATA_EXPORT_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/export
DATA_CONV_USER_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/park
DATA_CONV_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/converted
MODEL_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/model
RESULT_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/result
WORKING_DIR :  /content/gdrive/My Drive/weather_bigcontest/4_code/park


In [ ]:
# 데이터 무엇이 있는지 확인하자.
# 아래 코드를 수행하게 되면, "../input/"에서 리눅스 명령어 "ls"를 수행한 결과를 보여주게 된다. 이 때 나온 binary code를 "utf8"로 디코딩해서 보여준다.
from subprocess import check_output
print("===== Raw Data List =====\n", check_output(["ls", DATA_RAW_PATH]).decode("utf8"))
print("===== Export Data List =====\n" ,check_output(["ls", DATA_EXPORT_PATH]).decode("utf8"))
print("===== Converted User Data List =====\n" ,check_output(["ls", DATA_CONV_USER_PATH]).decode("utf8"))
print("===== Converted Data List =====\n" ,check_output(["ls", DATA_CONV_PATH]).decode("utf8"))
print("===== Model List =====\n" ,check_output(["ls", MODEL_PATH]).decode("utf8"))
print("===== Result file List =====\n" ,check_output(["ls", RESULT_PATH]).decode("utf8"))

===== Raw Data List =====
 plant1_train.csv
plant2_train.csv
plant2_train.gsheet
plant_test.csv
submit.csv
submit.gsheet

===== Export Data List =====
 dangjin_aws
seosan_asos
sinpyeong_aws
songak_forecast
songak_local
songsan_forecast
songsan_local

===== Converted User Data List =====
 backup_0.96model
data_all.csv
data_all_out.csv
data_all_out_in.csv
date_trend.csv
date_trend_point.csv
ExtraTreesClassifier_th_10.csv
GradientBoostingClassifier_all_feature_th_40.csv
lastplus_th_10.csv
last_th_08.csv
last_th_10.csv
model
model_cond_all.pkl
model_cond_all_plus_.pkl
model_hum_in.pkl
model_hum_out_loc.pkl
model_tem_coil.pkl
model_tem_in.pkl
model_tem_out_loc.pkl
plant1_by_loc.csv
plant1_by_loc.gsheet
plant2_by_loc.csv
plant_test_all_weather_feature_add_cluster_add_out.csv
plant_test_all_weather_feature_add_cluster_add_out_in.csv
plant_test_all_weather_feature_add_cluster_add_out_in.gsheet
plant_train_all_weather_feature_add_cluster.csv
predict.csv
RandomForestClassifier_th_10.csv
result.c

# 4. Import Library

In [ ]:
# 기본 데이터 라이브러리 로드
import sys # 시스템 파라미터에 접근할 수 있게 도와준다.
print("Python version: {}". format(sys.version))

import pandas as pd # 데이터 정제에 도움을 주는 라이브러리
print("pandas version: {}". format(pd.__version__))

import matplotlib # 매트랩에서 사용하는 시각화 도구를 사용할 수 있게 도와주는 시각화 도구
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np # 행렬 계산을 위해 필요한 라이브러리
print("NumPy version: {}". format(np.__version__))

import scipy as sp # 수학 관련 함수가 내장된 라이브러리
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display # 주피터 노트북에서 예쁘게 시각화 해주는 도구
print("IPython version: {}". format(IPython.__version__)) 

import sklearn # 각종 통계 도구와 머신 러닝 알고리즘이 내장되어 있는 라이브러리
print("scikit-learn version: {}". format(sklearn.__version__))

# 파이썬 내장 라이브러리
import random
import datetime
import time
import os
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import copy


# Jupyter Notebook 이나 ipython 을 사용하다보면 향후 버전이 올라갈 때 변경될 사항 등을 알려주는 경고 메시지(warning message)를 뜨지 않게 해준다.
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
pandas version: 1.0.5
matplotlib version: 3.2.2
NumPy version: 1.18.5
SciPy version: 1.4.1
IPython version: 5.5.0
scikit-learn version: 0.22.2.post1
-------------------------


In [ ]:
# 모델링 라이브러리, 수학 계산 툴 로드
# 일반적인 모델링 라이브러리
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

# 모델링 시 헬퍼 함수들
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.externals import joblib

# interpolation 함수
import scipy.interpolate as spi

# pickle
import pickle

# 시각화 도구
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

# 시각화 도구 default 세팅
# 주피터 노트북에서 plot 결과를 볼 수 있게 해준다.
%matplotlib inline
mpl.style.use('ggplot') # matplotlib에서 plot되는 결과를 선택할 수 있다.
sns.set_style('white') # seaborn에서 사용할 style을 설정할 수 있다.
pylab.rcParams['figure.figsize'] = 12,8 # plot의 크기와 선 등의 기본 값을 설정할 수 있다.

# 시각화 Style 설정하기
# print(plt.style.available)
# ['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', 'tableau-colorblind10', '_classic_test']

plt.style.use(['fivethirtyeight'])
sns.set_style('darkgrid')

# 5. Read Raw Data

## 1. Load Weather Data

### 1. 사용하는 함수 정리

In [ ]:
import datetime
def stringToDatetime(df, colName='mea_ddhr'):
  """
  describe : string 을 datetime으로 변경한다.
  input = pd.Dataframe
  output = pd.Dataframe
  """
  if type(df[colName][0]) != str: # 이미 자료형이 datetime인 경우 return 한다.
    return df
  if len(df[colName][0].split(' ')[1].split(':')) == 2: # 날짜 정보에 초에 대한 정보가 없을 경우 초를 붙여주고 datetime으로 변경한다.
    df[colName] = df[colName].apply(lambda x : x + ':00' if x != 'nan' else np.nan)
  df[colName] = df[colName].apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') if x != 'nan' else np.nan)
  return df

def createDateTime(df, colName='mea_ddhr'):
  """
  describe : datetime의 정보에서 date 정보와 0시 기준 몇 분인지를 나타내는 time을 분리하여 추가한다.
  input = pd.Dataframe
  output = pd.Dataframe
  """
  if type(df[colName][0]) == "str":
    df[colName] = df[colName].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M"))
  df = stringToDatetime(df, colName)
  df['date'] = df[colName].apply(lambda x : x.date())
  df['time'] = df[colName].apply(lambda x : np.float64(x.hour * 60 + x.minute))
  return df

### 2. 기상 데이터 전처리

<column 명>
1.   mea_ddhr : 관측시각 (YYYY-MM-DD HH:MM:SS)
2.   ta : 기온(°C)
3.   hm : 습도(%)
3.   ws : 풍속(m/s)
4.   rn : 강수량(mm)
5.   dew : 이슬점 온도


#### 1. 당진 616 AWS 기록 (1시간)


In [ ]:
# 당진 616 AWS
AWS_616_2016 = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "dangjin_aws/AWS_616_2016.csv"), encoding = 'cp949')
AWS_616_2017 = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "dangjin_aws/AWS_616_2017.csv"), encoding = 'cp949')
AWS_616_2018 = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "dangjin_aws/AWS_616_2018.csv"), encoding = 'cp949')
AWS_616_2019 = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "dangjin_aws/AWS_616_2019.csv"), encoding = 'cp949')
AWS_616_2020 = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "dangjin_aws/AWS_616_2020.csv"), encoding = 'cp949')

In [ ]:
# 칼럼명 바꾸기
# 순서대로 지점번호 / 시간 / 기온 / 풍향 / 풍속 / 강수량 / 현지기압 / 해면기압 / 습도 / 일사량 / 일조량
AWS_616_2016.columns = ["stn_id", "mea_ddhr", "ta", "wd", "ws", "rn", "lp", "ap", "hm", "sr", "ss"]
AWS_616_2017.columns = ["stn_id", "mea_ddhr", "ta", "wd", "ws", "rn", "lp", "ap", "hm", "sr", "ss"]
AWS_616_2018.columns = ["stn_id", "mea_ddhr", "ta", "wd", "ws", "rn", "lp", "ap", "hm", "sr", "ss"]
AWS_616_2019.columns = ["stn_id", "mea_ddhr", "ta", "wd", "ws", "rn", "lp", "ap", "hm", "sr", "ss"]
AWS_616_2020.columns = ["stn_id", "mea_ddhr", "ta", "wd", "ws", "rn", "lp", "ap", "hm", "sr", "ss"]

In [ ]:
# 사용하지 못하는 변수 제거
# sr, ss는 7131개만 존재한다. 2017년 초반까지만 값을 기록한듯. 하지만 이마저도 다 값이 0.
# lp, ap는 105개만 값이 존재. 활용 불가.
AWS_616_2016 = AWS_616_2016.drop(["stn_id", "lp", "ap", "sr", "ss", "wd"], axis=1)
AWS_616_2017 = AWS_616_2017.drop(["stn_id", "lp", "ap", "sr", "ss", "wd"], axis=1)
AWS_616_2018 = AWS_616_2018.drop(["stn_id", "lp", "ap", "sr", "ss", "wd"], axis=1)
AWS_616_2019 = AWS_616_2019.drop(["stn_id", "lp", "ap", "sr", "ss", "wd"], axis=1)
AWS_616_2020 = AWS_616_2020.drop(["stn_id", "lp", "ap", "sr", "ss", "wd"], axis=1)

In [ ]:
# 당진 모든년도 병합
dangjin_weather = pd.concat([AWS_616_2016, AWS_616_2017, AWS_616_2018, AWS_616_2019, AWS_616_2020], ignore_index=True)

In [ ]:
# column 순서 재정렬
dangjin_weather = dangjin_weather[["mea_ddhr","ta", "hm", "ws", "rn"]]

In [ ]:
# Date, Time 변수 생성
dangjin_weather = createDateTime(dangjin_weather)

In [ ]:
dangjin_weather.name = "dangjin_weather"

In [ ]:
dangjin_weather.head(20)

,mea_ddhr,ta,hm,ws,rn,date,time
0,2016-04-01 00:00:00,6.8,41.2,0.3,0.0,2016-04-01,0.0
1,2016-04-01 01:00:00,5.9,43.8,0.3,0.0,2016-04-01,60.0
2,2016-04-01 02:00:00,5.5,45.5,0.3,0.0,2016-04-01,120.0
3,2016-04-01 03:00:00,5.2,52.1,0.2,0.0,2016-04-01,180.0
4,2016-04-01 04:00:00,4.7,47.1,0.0,0.0,2016-04-01,240.0
5,2016-04-01 05:00:00,5.5,43.7,0.0,0.0,2016-04-01,300.0
6,2016-04-01 06:00:00,6.8,40.9,1.0,0.0,2016-04-01,360.0
7,2016-04-01 07:00:00,7.5,39.3,1.1,0.0,2016-04-01,420.0
8,2016-04-01 08:00:00,10.6,31.5,1.3,0.0,2016-04-01,480.0
9,2016-04-01 09:00:00,13.9,28.9,0.6,0.0,2016-04-01,540.0


In [ ]:
dangjin_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34893 entries, 0 to 34892
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   mea_ddhr  34893 non-null  datetime64[ns]
 1   ta        34822 non-null  float64       
 2   hm        34863 non-null  float64       
 3   ws        34825 non-null  float64       
 4   rn        34780 non-null  float64       
 5   date      34893 non-null  object        
 6   time      34893 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 1.9+ MB


In [ ]:
dangjin_weather.describe()

,ta,hm,ws,rn,time
count,34822.000000,34863.000000,34825.000000,34780.000000,34893.000000
mean,12.430248,74.978516,1.485976,0.104155,690.100593
std,10.365760,19.063629,1.145384,0.852400,415.441826
min,-16.600000,7.600000,0.000000,0.000000,0.000000
25%,3.600000,61.400000,0.600000,0.000000,300.000000
50%,13.000000,79.800000,1.300000,0.000000,720.000000
75%,21.200000,93.000000,2.100000,0.000000,1080.000000
max,37.100000,98.900000,8.300000,67.000000,1380.000000


#### 2. 신평 637 AWS 기록 (1시간)

In [ ]:
# 신평 637 AWS
AWS_637_2016 = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "sinpyeong_aws/AWS_637_2016.csv"), encoding = 'cp949')
AWS_637_2017 = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "sinpyeong_aws/AWS_637_2017.csv"), encoding = 'cp949')
AWS_637_2018 = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "sinpyeong_aws/AWS_637_2018.csv"), encoding = 'cp949')
AWS_637_2019 = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "sinpyeong_aws/AWS_637_2019.csv"), encoding = 'cp949')
AWS_637_2020 = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "sinpyeong_aws/AWS_637_2020.csv"), encoding = 'cp949')

In [ ]:
# 칼럼명 바꾸기
# 순서대로 지점번호 / 시간 / 기온 / 풍향 / 풍속 / 강수량 / 현지기압 / 해면기압 / 습도 / 일사량 / 일조량
AWS_637_2016.columns = ["stn_id", "mea_ddhr", "ta", "wd", "ws", "rn", "lp", "ap", "hm", "sr", "ss"]
AWS_637_2017.columns = ["stn_id", "mea_ddhr", "ta", "wd", "ws", "rn", "lp", "ap", "hm", "sr", "ss"]
AWS_637_2018.columns = ["stn_id", "mea_ddhr", "ta", "wd", "ws", "rn", "lp", "ap", "hm", "sr", "ss"]
AWS_637_2019.columns = ["stn_id", "mea_ddhr", "ta", "wd", "ws", "rn", "lp", "ap", "hm", "sr", "ss"]
AWS_637_2020.columns = ["stn_id", "mea_ddhr", "ta", "wd", "ws", "rn", "lp", "ap", "hm", "sr", "ss"]

In [ ]:
# 사용하지 못하는 변수 제거
# hm은 2019년 부터 자료가 있어 사용하지 않기로 했다.
AWS_637_2016 = AWS_637_2016.drop(["stn_id", "sr", "ss", "wd", "hm"], axis=1)
AWS_637_2017 = AWS_637_2017.drop(["stn_id", "sr", "ss", "wd", "hm"], axis=1)
AWS_637_2018 = AWS_637_2018.drop(["stn_id", "sr", "ss", "wd", "hm"], axis=1)
AWS_637_2019 = AWS_637_2019.drop(["stn_id", "sr", "ss", "wd", "hm"], axis=1)
AWS_637_2020 = AWS_637_2020.drop(["stn_id", "sr", "ss", "wd", "hm"], axis=1)

In [ ]:
# 신평 모든년도 병합
sinpyeong_weather = pd.concat([AWS_637_2016, AWS_637_2017, AWS_637_2018, AWS_637_2019, AWS_637_2020], ignore_index=True)

In [ ]:
# column 순서 재정렬
sinpyeong_weather = sinpyeong_weather[["mea_ddhr","ta", "ws", "rn", "lp", "ap"]]

In [ ]:
# date, time 변수 추가
sinpyeong_weather = createDateTime(sinpyeong_weather)

In [ ]:
sinpyeong_weather.name = "sinpyeong_weather"

In [ ]:
# 2019년에만 hum 정보가 있다.
sinpyeong_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34980 entries, 0 to 34979
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   mea_ddhr  34980 non-null  datetime64[ns]
 1   ta        34933 non-null  float64       
 2   ws        34947 non-null  float64       
 3   rn        34851 non-null  float64       
 4   lp        34964 non-null  float64       
 5   ap        34964 non-null  float64       
 6   date      34980 non-null  object        
 7   time      34980 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 2.1+ MB


In [ ]:
sinpyeong_weather.describe()

,ta,ws,rn,lp,ap,time
count,34933.000000,34947.000000,34851.000000,34964.000000,34964.000000,34980.000000
mean,12.928071,1.987647,0.109566,1014.374488,1016.919663,690.370497
std,10.701824,1.343596,0.935056,8.375025,8.457192,415.307228
min,-16.400000,0.000000,0.000000,986.100000,988.600000,0.000000
25%,3.800000,1.000000,0.000000,1007.800000,1010.200000,360.000000
50%,13.600000,1.700000,0.000000,1014.400000,1016.900000,720.000000
75%,22.000000,2.700000,0.000000,1020.900000,1023.500000,1080.000000
max,39.000000,10.900000,52.500000,1037.300000,1039.900000,1380.000000


#### 3. 서산 ASOS 기록 (1시간)

In [ ]:
# 당진 616 AWS
ASOS_129_2016 = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "seosan_asos/ASOS_129_2016_new.csv"), encoding = 'cp949')
ASOS_129_2017 = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "seosan_asos/ASOS_129_2017_new.csv"), encoding = 'cp949')
ASOS_129_2018 = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "seosan_asos/ASOS_129_2018_new.csv"), encoding = 'cp949')
ASOS_129_2019 = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "seosan_asos/ASOS_129_2019_new.csv"), encoding = 'cp949')
ASOS_129_2020 = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "seosan_asos/ASOS_129_2020_new.csv"), encoding = 'cp949')

In [ ]:
ASOS_129_2019.columns

Index(['지점', '지점명', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '증기압(hPa)',
       '이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '일조(hr)', '일사(MJ/m2)',
       '지면온도(°C)'],
      dtype='object')

In [ ]:
# 칼럼명 바꾸기"lp", "ap", "hm", "sr", "ss"
# 순서대로                 지점번호  / 지점명 /    일시    / 기온 / 강수 / 풍속 / 습도 / 증기압 / 이슬점온도 / 현지기압 / 해면기압 / 일조 / 일사 / 지면온도
ASOS_129_2016.columns = ["stn_id", "loc", "mea_ddhr", "ta", "rn", "ws", "hm", "sp", "dew", "lp", "ap", "sr", "ss", "ground"]
ASOS_129_2017.columns = ["stn_id", "loc", "mea_ddhr", "ta", "rn", "ws", "hm", "sp", "dew", "lp", "ap", "sr", "ss", "ground"]
ASOS_129_2018.columns = ["stn_id", "loc", "mea_ddhr", "ta", "rn", "ws", "hm", "sp", "dew", "lp", "ap", "sr", "ss", "ground"]
ASOS_129_2019.columns = ["stn_id", "loc", "mea_ddhr", "ta", "rn", "ws", "hm", "sp", "dew", "lp", "ap", "sr", "ss", "ground"]
ASOS_129_2020.columns = ["stn_id", "loc", "mea_ddhr", "ta", "rn", "ws", "hm", "sp", "dew", "lp", "ap", "sr", "ss", "ground"]

In [ ]:
# 사용하지 못하는 변수 제거
ASOS_129_2016 = ASOS_129_2016.drop(["stn_id", "loc"], axis=1)
ASOS_129_2017 = ASOS_129_2017.drop(["stn_id", "loc"], axis=1)
ASOS_129_2018 = ASOS_129_2018.drop(["stn_id", "loc"], axis=1)
ASOS_129_2019 = ASOS_129_2019.drop(["stn_id", "loc"], axis=1)
ASOS_129_2020 = ASOS_129_2020.drop(["stn_id", "loc"], axis=1)

In [ ]:
# 서산 모든년도 병합
seosan_weather = pd.concat([ASOS_129_2016, ASOS_129_2017, ASOS_129_2018, ASOS_129_2019, ASOS_129_2020], ignore_index=True)

In [ ]:
# column 순서 재정의
seosan_weather = seosan_weather[["mea_ddhr", "ta", "hm", "ws", "rn", "dew", "sp",	"lp",	"ap",	"sr",	"ss",	"ground"]]

In [ ]:
# Date, Time 변수 생성
seosan_weather = createDateTime(seosan_weather)

In [ ]:
seosan_weather.name = "seosan_weather"

In [ ]:
seosan_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34993 entries, 0 to 34992
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   mea_ddhr  34993 non-null  datetime64[ns]
 1   ta        34991 non-null  float64       
 2   hm        34989 non-null  float64       
 3   ws        34987 non-null  float64       
 4   rn        3032 non-null   float64       
 5   dew       34989 non-null  float64       
 6   sp        34989 non-null  float64       
 7   lp        34989 non-null  float64       
 8   ap        34989 non-null  float64       
 9   sr        19080 non-null  float64       
 10  ss        19080 non-null  float64       
 11  ground    34910 non-null  float64       
 12  date      34993 non-null  object        
 13  time      34993 non-null  float64       
dtypes: datetime64[ns](1), float64(12), object(1)
memory usage: 3.7+ MB


In [ ]:
seosan_weather.describe()

,ta,hm,ws,rn,dew,sp,lp,ap,sr,ss,ground,time
count,34991.000000,34989.000000,34987.000000,3032.000000,34989.000000,34989.000000,34989.000000,34989.000000,19080.000000,19080.000000,34910.000000,34993.000000
mean,12.677806,75.643159,1.928193,1.295580,7.888522,13.442390,1013.747072,1017.081051,0.533475,1.012751,15.348445,690.124311
std,10.488240,20.191221,1.528140,3.432745,11.224211,9.225442,8.292580,8.396465,0.447833,0.908161,11.824762,415.353003
min,-15.600000,10.000000,0.000000,0.000000,-22.300000,1.000000,983.800000,986.600000,0.000000,0.000000,-9.300000,0.000000
25%,3.800000,60.000000,0.700000,0.000000,-1.500000,5.500000,1007.200000,1010.400000,0.000000,0.210000,4.700000,360.000000
50%,13.400000,79.000000,1.600000,0.200000,8.400000,11.000000,1013.800000,1017.100000,0.600000,0.790000,15.200000,720.000000
75%,21.600000,95.000000,2.900000,1.200000,17.500000,20.000000,1020.200000,1023.600000,1.000000,1.660000,24.300000,1080.000000
max,36.800000,100.000000,11.700000,102.700000,30.200000,42.900000,1036.300000,1039.700000,1.000000,4.850000,59.700000,1380.000000


#### 4. 송산 동네 기록 (1시간)

In [ ]:
def localWeatherToDf(date_start, date_end, dataList):
  """
  describe : string 을 datetime으로 변경한다.
  <input>
  date_start - 시작 날짜 (datetime)
  date_end - 끝 날짜 (datetime)
  dataList - 합칠 dataframe 목록 (list)

  <output>
  pd.Dataframe
  """
  ret_df = pd.DataFrame(columns=["mea_ddhr", "ta", "hm", "ws", "rn"])
  colName = ["ta", "hm", "ws", "rn"]
  date = date_start
  mea_date = date
  month_checker = False

  idx = 0
  for i in range(dataList[0].shape[0]):
    if i%500 == 0:
      print(f'{mea_date} 작업중 ...')

    num = len(dataList[0].loc[i, "day"].split(":"))

    if num == 1:
      ret_df.loc[idx] = 0
      mea_date = date + timedelta(days=(int(float(dataList[0].loc[i, "day"]))-1), hours=int(float(dataList[0].loc[i, "hour"]/100)))
      ret_df.loc[idx, "mea_ddhr"] = mea_date
      for j in range(len(colName)):
        ret_df.loc[idx, colName[j]] = float(dataList[j].loc[i, "value"])
      idx += 1
    else:
      month_checker = True

    if (month_checker == True):
      date += relativedelta(months=1)
      month_checker = False

  
  ret_df["ta"] = ret_df["ta"].apply(lambda x : np.float(x))
  ret_df["hm"] = ret_df["hm"].apply(lambda x : np.float(x))
  ret_df["ws"] = ret_df["ws"].apply(lambda x : np.float(x))
  ret_df["rn"] = ret_df["rn"].apply(lambda x : np.float(x))
  return ret_df


In [ ]:
def localWeatherToDf_INT(date_start, date_end, dataList):
  """
  describe : string 을 datetime으로 변경한다.
  <input>
  date_start - 시작 날짜 (datetime)
  date_end - 끝 날짜 (datetime)
  dataList - 합칠 dataframe 목록 (list)

  <output>
  pd.Dataframe
  """
  ret_df = pd.DataFrame(columns=["mea_ddhr", "ta", "hm", "ws", "rn"])
  colName = ["ta", "hm", "ws", "rn"]
  date = date_start
  mea_date = date
  month_checker = False

  idx = 0
  for i in range(dataList[0].shape[0]):
    if i%500 == 0:
      print(f'{mea_date} 작업중 ...')

    num = len(dataList[0].loc[i, "day"].split(":"))

    if num == 1:
      ret_df.loc[idx] = 0
      mea_date = date + timedelta(days=(int(float(dataList[0].loc[i, "day"]))-1), hours=int(float(dataList[0].loc[i, "hour"]/100)))
      ret_df.loc[idx, "mea_ddhr"] = mea_date
      for j in range(len(colName)):
        ret_df.loc[idx, colName[j]] = float(dataList[j].loc[i, "value"])
      idx += 1
    else:
      month_checker = True

    if (month_checker == True):
      date += relativedelta(months=1)
      month_checker = False

  
  ret_df["ta"] = ret_df["ta"].apply(lambda x : np.int(x))
  ret_df["hm"] = ret_df["hm"].apply(lambda x : np.int(x))
  ret_df["ws"] = ret_df["ws"].apply(lambda x : np.int(x))
  ret_df["rn"] = ret_df["rn"].apply(lambda x : np.int(x))
  return ret_df


In [ ]:
# 송산 2016 3월 동네 기록 데이터
songsan_2016_03_ta = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2016_03_ta_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2016_03_hm = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2016_03_hm_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2016_03_ws = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2016_03_ws_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2016_03_rn = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2016_03_rn_songsan.csv"), encoding = 'ISO-8859-1')

# 송산 2016 동네 기록 데이터
songsan_2016_ta = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2016_ta_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2016_hm = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2016_hm_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2016_ws = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2016_ws_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2016_rn = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2016_rn_songsan.csv"), encoding = 'ISO-8859-1')

# 송산 2017 동네 기록 데이터
songsan_2017_ta = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2017_ta_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2017_hm = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2017_hm_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2017_ws = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2017_ws_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2017_rn = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2017_rn_songsan.csv"), encoding = 'ISO-8859-1')

# 송산 2018 동네 기록 데이터
songsan_2018_ta = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2018_ta_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2018_hm = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2018_hm_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2018_ws = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2018_ws_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2018_rn = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2018_rn_songsan.csv"), encoding = 'ISO-8859-1')

# 송산 2019 동네 기록 데이터
songsan_2019_ta = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2019_ta_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2019_hm = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2019_hm_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2019_ws = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2019_ws_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2019_rn = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2019_rn_songsan.csv"), encoding = 'ISO-8859-1')

# 송산 2020 동네 기록 데이터
songsan_2020_ta = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2020_ta_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2020_hm = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2020_hm_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2020_ws = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2020_ws_songsan.csv"), encoding = 'ISO-8859-1')
songsan_2020_rn = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songsan_local/2020_rn_songsan.csv"), encoding = 'ISO-8859-1')


In [ ]:
# 변수명 바꿔주기
songsan_2016_03_ta.columns = ["day", "hour", "value"]
songsan_2016_03_hm.columns = ["day", "hour", "value"]
songsan_2016_03_ws.columns = ["day", "hour", "value"]
songsan_2016_03_rn.columns = ["day", "hour", "value"]

songsan_2016_ta.columns = ["day", "hour", "value"]
songsan_2016_hm.columns = ["day", "hour", "value"]
songsan_2016_ws.columns = ["day", "hour", "value"]
songsan_2016_rn.columns = ["day", "hour", "value"]

songsan_2017_ta.columns = ["day", "hour", "value"]
songsan_2017_hm.columns = ["day", "hour", "value"]
songsan_2017_ws.columns = ["day", "hour", "value"]
songsan_2017_rn.columns = ["day", "hour", "value"]

songsan_2018_ta.columns = ["day", "hour", "value"]
songsan_2018_hm.columns = ["day", "hour", "value"]
songsan_2018_ws.columns = ["day", "hour", "value"]
songsan_2018_rn.columns = ["day", "hour", "value"]

songsan_2019_ta.columns = ["day", "hour", "value"]
songsan_2019_hm.columns = ["day", "hour", "value"]
songsan_2019_ws.columns = ["day", "hour", "value"]
songsan_2019_rn.columns = ["day", "hour", "value"]

songsan_2020_ta.columns = ["day", "hour", "value"]
songsan_2020_hm.columns = ["day", "hour", "value"]
songsan_2020_ws.columns = ["day", "hour", "value"]
songsan_2020_rn.columns = ["day", "hour", "value"]

In [ ]:
songsan_2016_03_ta

,day,hour,value
0,1,0.0,-2.9
1,1,100.0,-1.4
2,1,200.0,-0.2
3,1,300.0,0.6
4,1,400.0,1.6
...,...,...,...
740,31,2000.0,4.3
741,31,2100.0,4.9
742,31,2200.0,6.1
743,31,2300.0,9.9


In [ ]:
# 마지막 행 Nan 제거
songsan_2016_03_ta = songsan_2016_03_ta.drop(songsan_2016_03_ta.shape[0]-1)
songsan_2016_03_hm = songsan_2016_03_hm.drop(songsan_2016_03_hm.shape[0]-1)
songsan_2016_03_ws = songsan_2016_03_ws.drop(songsan_2016_03_ws.shape[0]-1)
songsan_2016_03_rn = songsan_2016_03_rn.drop(songsan_2016_03_rn.shape[0]-1)
print(songsan_2016_03_ta.shape)
print(songsan_2016_03_hm.shape)
print(songsan_2016_03_ws.shape)
print(songsan_2016_03_rn.shape)

(744, 3)
(744, 3)
(744, 3)
(744, 3)


In [ ]:
songsan_2016_ta

,day,hour,value
0,1,0.0,14.000000
1,1,100.0,16.299999
2,1,200.0,18.299999
3,1,300.0,19.600000
4,1,400.0,20.600000
...,...,...,...
6603,31,1900.0,1.200000
6604,31,2000.0,0.600000
6605,31,2100.0,0.800000
6606,31,2200.0,0.800000


In [ ]:
# 송산 기존 데이터 첫행 데이터 추가
# Start : 20160401

data = {'day':["Start : 20160401"], 'hour': [0], 'value': [0]}
temp = pd.DataFrame(data)
temp

,day,hour,value
0,Start : 20160401,0,0


In [ ]:
# 송산 2016년도 병합

songsan_2016_ta = pd.concat([songsan_2016_03_ta, temp, songsan_2016_ta], ignore_index=True)
songsan_2016_hm = pd.concat([songsan_2016_03_hm, temp, songsan_2016_hm], ignore_index=True)
songsan_2016_rn = pd.concat([songsan_2016_03_rn, temp, songsan_2016_rn], ignore_index=True)
songsan_2016_ws = pd.concat([songsan_2016_03_ws, temp, songsan_2016_ws], ignore_index=True)
songsan_2016_ta

# songsan_2016_ta = pd.concat([songsan_2016_03_ta, songsan_2016_ta], ignore_index=True)
# songsan_2016_hm = pd.concat([songsan_2016_03_hm, songsan_2016_hm], ignore_index=True)
# songsan_2016_rn = pd.concat([songsan_2016_03_rn, songsan_2016_rn], ignore_index=True)
# songsan_2016_ws = pd.concat([songsan_2016_03_ws, songsan_2016_ws], ignore_index=True)
# songsan_2016_ta

,day,hour,value
0,1,0.0,-2.9
1,1,100.0,-1.4
2,1,200.0,-0.2
3,1,300.0,0.6
4,1,400.0,1.6
...,...,...,...
7348,31,1900.0,1.2
7349,31,2000.0,0.6
7350,31,2100.0,0.8
7351,31,2200.0,0.8


In [ ]:
songsan_2016_ta[742:750]

,day,hour,value
742,31,2200.0,6.100000
743,31,2300.0,9.900000
744,Start : 20160401,0.0,0.000000
745,1,0.0,14.000000
746,1,100.0,16.299999
747,1,200.0,18.299999
748,1,300.0,19.600000
749,1,400.0,20.600000


In [ ]:
# 4개의 정보의 크기가 같은지 확인
print("====== 송산 2016 ======")
print(songsan_2016_ta.shape)
print(songsan_2016_hm.shape)
print(songsan_2016_ws.shape)
print(songsan_2016_rn.shape)

print("====== 송산 2017 ======")
print(songsan_2017_ta.shape)
print(songsan_2017_hm.shape)
print(songsan_2017_ws.shape)
print(songsan_2017_rn.shape)

print("====== 송산 2018 ======")
print(songsan_2018_ta.shape)
print(songsan_2018_hm.shape)
print(songsan_2018_ws.shape)
print(songsan_2018_rn.shape)

print("====== 송산 2019 ======")
print(songsan_2019_ta.shape)
print(songsan_2019_hm.shape)
print(songsan_2019_ws.shape)
print(songsan_2019_rn.shape)

print("====== 송산 2020 ======")
print(songsan_2020_ta.shape)
print(songsan_2020_hm.shape)
print(songsan_2020_ws.shape)
print(songsan_2020_rn.shape)

====== 송산 2016 ======
(7353, 3)
(7353, 3)
(7353, 3)
(7353, 3)
====== 송산 2017 ======
(8771, 3)
(8771, 3)
(8771, 3)
(8771, 3)
====== 송산 2018 ======
(8771, 3)
(8771, 3)
(8771, 3)
(8771, 3)
====== 송산 2019 ======
(8771, 3)
(8771, 3)
(8771, 3)
(8771, 3)
====== 송산 2020 ======
(2186, 3)
(2186, 3)
(2186, 3)
(2186, 3)


In [ ]:
songsan_2019_hm.tail()

,day,hour,value
8766,31,1900.0,86.0
8767,31,2000.0,92.0
8768,31,2100.0,94.0
8769,31,2200.0,91.0
8770,31,2300.0,88.0


In [ ]:
# 송산 2016 병합
from datetime import datetime
date_start = datetime(2016, 3, 1, 0, 0, 0)
date_end = datetime(2016, 12, 31, 23, 0, 0)
dataList = [songsan_2016_ta, songsan_2016_hm, songsan_2016_ws, songsan_2016_rn]
songsan_2016 = localWeatherToDf(date_start, date_end, dataList)
songsan_2016

2016-03-01 00:00:00 작업중 ...
2016-03-21 19:00:00 작업중 ...
2016-04-11 14:00:00 작업중 ...
2016-05-02 09:00:00 작업중 ...
2016-05-23 05:00:00 작업중 ...
2016-06-13 00:00:00 작업중 ...
2016-07-03 19:00:00 작업중 ...
2016-07-24 15:00:00 작업중 ...
2016-08-14 10:00:00 작업중 ...
2016-09-04 05:00:00 작업중 ...
2016-09-25 01:00:00 작업중 ...
2016-10-15 20:00:00 작업중 ...
2016-11-05 15:00:00 작업중 ...
2016-11-26 11:00:00 작업중 ...
2016-12-17 06:00:00 작업중 ...


,mea_ddhr,ta,hm,ws,rn
0,2016-03-01 00:00:00,-2.9,63.0,1.0,0.0
1,2016-03-01 01:00:00,-1.4,57.0,0.4,0.0
2,2016-03-01 02:00:00,-0.2,53.0,0.8,0.0
3,2016-03-01 03:00:00,0.6,50.0,1.1,0.0
4,2016-03-01 04:00:00,1.6,48.0,1.2,0.0
...,...,...,...,...,...
7339,2016-12-31 19:00:00,1.2,95.0,0.6,0.0
7340,2016-12-31 20:00:00,0.6,95.0,0.5,0.0
7341,2016-12-31 21:00:00,0.8,92.0,0.1,0.0
7342,2016-12-31 22:00:00,0.8,91.0,0.4,0.0


In [ ]:
# type(songsan_weather["date"][0])

In [ ]:
# import datetime
# from dateutil.parser import parse
# # print(parse('2016-04-16').date())

# songsan_2016 = createDateTime(songsan_2016)
# songsan_2016[songsan_2016["date"] == parse('2016-05-01').date()]

In [ ]:
# 송산 2017 병합
from datetime import datetime
date_start = datetime(2017, 1, 1, 0, 0, 0)
date_end = datetime(2017, 12, 31, 23, 0, 0)
dataList = [songsan_2017_ta, songsan_2017_hm, songsan_2017_ws, songsan_2017_rn]
songsan_2017 = localWeatherToDf(date_start, date_end, dataList)
songsan_2017

2017-01-01 00:00:00 작업중 ...
2017-01-21 19:00:00 작업중 ...
2017-02-11 14:00:00 작업중 ...
2017-03-04 09:00:00 작업중 ...
2017-03-25 05:00:00 작업중 ...
2017-04-15 00:00:00 작업중 ...
2017-05-05 19:00:00 작업중 ...
2017-05-26 15:00:00 작업중 ...
2017-06-16 10:00:00 작업중 ...
2017-07-07 05:00:00 작업중 ...
2017-07-28 01:00:00 작업중 ...
2017-08-17 20:00:00 작업중 ...
2017-09-07 15:00:00 작업중 ...
2017-09-28 11:00:00 작업중 ...
2017-10-19 06:00:00 작업중 ...
2017-11-09 01:00:00 작업중 ...
2017-11-29 21:00:00 작업중 ...
2017-12-20 16:00:00 작업중 ...


,mea_ddhr,ta,hm,ws,rn
0,2017-01-01 00:00:00,1.9,90.0,1.6,0.0
1,2017-01-01 01:00:00,3.5,85.0,0.5,0.0
2,2017-01-01 02:00:00,4.9,80.0,0.7,0.0
3,2017-01-01 03:00:00,6.7,70.0,1.1,0.0
4,2017-01-01 04:00:00,8.1,67.0,1.9,0.0
...,...,...,...,...,...
8755,2017-12-31 19:00:00,-4.9,82.0,1.6,0.0
8756,2017-12-31 20:00:00,-4.6,76.0,2.0,0.0
8757,2017-12-31 21:00:00,-4.6,75.0,2.3,0.0
8758,2017-12-31 22:00:00,-4.7,74.0,2.0,0.0


In [ ]:
# 송산 2018 병합
from datetime import datetime
date_start = datetime(2018, 1, 1, 0, 0, 0)
date_end = datetime(2018, 12, 31, 23, 0, 0)
dataList = [songsan_2018_ta, songsan_2018_hm, songsan_2018_ws, songsan_2018_rn]
songsan_2018 = localWeatherToDf(date_start, date_end, dataList)
songsan_2018

2018-01-01 00:00:00 작업중 ...
2018-01-21 19:00:00 작업중 ...
2018-02-11 14:00:00 작업중 ...
2018-03-04 09:00:00 작업중 ...
2018-03-25 05:00:00 작업중 ...
2018-04-15 00:00:00 작업중 ...
2018-05-05 19:00:00 작업중 ...
2018-05-26 15:00:00 작업중 ...
2018-06-16 10:00:00 작업중 ...
2018-07-07 05:00:00 작업중 ...
2018-07-28 01:00:00 작업중 ...
2018-08-17 20:00:00 작업중 ...
2018-09-07 15:00:00 작업중 ...
2018-09-28 11:00:00 작업중 ...
2018-10-19 06:00:00 작업중 ...
2018-11-09 01:00:00 작업중 ...
2018-11-29 21:00:00 작업중 ...
2018-12-20 16:00:00 작업중 ...


,mea_ddhr,ta,hm,ws,rn
0,2018-01-01 00:00:00,0.9,55.0,1.3,0.0
1,2018-01-01 01:00:00,2.0,44.0,1.1,0.0
2,2018-01-01 02:00:00,3.2,37.0,0.6,0.0
3,2018-01-01 03:00:00,3.5,37.0,0.9,0.0
4,2018-01-01 04:00:00,3.1,37.0,0.7,0.0
...,...,...,...,...,...
8755,2018-12-31 19:00:00,-3.7,74.0,1.1,0.0
8756,2018-12-31 20:00:00,-5.6,70.0,0.2,0.0
8757,2018-12-31 21:00:00,-5.5,78.0,0.3,0.0
8758,2018-12-31 22:00:00,-4.8,83.0,0.1,0.0


In [ ]:
# 송산 2019 병합
from datetime import datetime
date_start = datetime(2019, 1, 1, 0, 0, 0)
date_end = datetime(2019, 12, 31, 23, 0, 0)
dataList = [songsan_2019_ta, songsan_2019_hm, songsan_2019_ws, songsan_2019_rn]
songsan_2019 = localWeatherToDf(date_start, date_end, dataList)
songsan_2019

2019-01-01 00:00:00 작업중 ...
2019-01-21 19:00:00 작업중 ...
2019-02-11 14:00:00 작업중 ...
2019-03-04 09:00:00 작업중 ...
2019-03-25 05:00:00 작업중 ...
2019-04-15 00:00:00 작업중 ...
2019-05-05 19:00:00 작업중 ...
2019-05-26 15:00:00 작업중 ...
2019-06-16 10:00:00 작업중 ...
2019-07-07 05:00:00 작업중 ...
2019-07-28 01:00:00 작업중 ...
2019-08-17 20:00:00 작업중 ...
2019-09-07 15:00:00 작업중 ...
2019-09-28 11:00:00 작업중 ...
2019-10-19 06:00:00 작업중 ...
2019-11-09 01:00:00 작업중 ...
2019-11-29 21:00:00 작업중 ...
2019-12-20 16:00:00 작업중 ...


,mea_ddhr,ta,hm,ws,rn
0,2019-01-01 00:00:00,-3.0,70.0,0.6,0.0
1,2019-01-01 01:00:00,-2.0,71.0,1.3,0.0
2,2019-01-01 02:00:00,-0.7,70.0,1.4,0.0
3,2019-01-01 03:00:00,-1.1,80.0,1.6,0.0
4,2019-01-01 04:00:00,-2.1,91.0,1.3,0.0
...,...,...,...,...,...
8755,2019-12-31 19:00:00,-4.5,86.0,0.7,0.0
8756,2019-12-31 20:00:00,-4.1,92.0,1.0,0.0
8757,2019-12-31 21:00:00,-3.2,94.0,0.4,0.0
8758,2019-12-31 22:00:00,-3.0,91.0,0.3,0.0


In [ ]:
# 송산 2020 병합
from datetime import datetime
date_start = datetime(2020, 1, 1, 0, 0, 0)
date_end = datetime(2020, 3, 31, 23, 0, 0)
dataList = [songsan_2020_ta, songsan_2020_hm, songsan_2020_ws, songsan_2020_rn]
songsan_2020 = localWeatherToDf(date_start, date_end, dataList)
songsan_2020

2020-01-01 00:00:00 작업중 ...
2020-01-21 19:00:00 작업중 ...
2020-02-11 14:00:00 작업중 ...
2020-03-03 09:00:00 작업중 ...
2020-03-24 05:00:00 작업중 ...


,mea_ddhr,ta,hm,ws,rn
0,2020-01-01 00:00:00,-0.2,81.0,1.1,0.0
1,2020-01-01 01:00:00,2.4,64.0,0.7,0.0
2,2020-01-01 02:00:00,2.0,69.0,0.8,0.0
3,2020-01-01 03:00:00,1.2,72.0,0.7,0.0
4,2020-01-01 04:00:00,1.3,72.0,0.6,0.0
...,...,...,...,...,...
2179,2020-03-31 19:00:00,8.5,58.0,1.4,0.0
2180,2020-03-31 20:00:00,8.3,56.0,1.1,0.0
2181,2020-03-31 21:00:00,8.4,51.0,1.3,0.0
2182,2020-03-31 22:00:00,9.4,47.0,1.7,0.0


In [ ]:
# 송산 모든년도 병합
songsan_weather = pd.concat([songsan_2016, songsan_2017, songsan_2018, songsan_2019, songsan_2020], ignore_index=True)
songsan_weather

,mea_ddhr,ta,hm,ws,rn
0,2016-03-01 00:00:00,-2.9,63.0,1.0,0.0
1,2016-03-01 01:00:00,-1.4,57.0,0.4,0.0
2,2016-03-01 02:00:00,-0.2,53.0,0.8,0.0
3,2016-03-01 03:00:00,0.6,50.0,1.1,0.0
4,2016-03-01 04:00:00,1.6,48.0,1.2,0.0
...,...,...,...,...,...
35803,2020-03-31 19:00:00,8.5,58.0,1.4,0.0
35804,2020-03-31 20:00:00,8.3,56.0,1.1,0.0
35805,2020-03-31 21:00:00,8.4,51.0,1.3,0.0
35806,2020-03-31 22:00:00,9.4,47.0,1.7,0.0


In [ ]:
# date, time 변수 제작
import datetime
songsan_weather_utc = createDateTime(songsan_weather)

In [ ]:
songsan_weather_utc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35808 entries, 0 to 35807
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   mea_ddhr  35808 non-null  object 
 1   ta        35808 non-null  float64
 2   hm        35808 non-null  float64
 3   ws        35808 non-null  float64
 4   rn        35808 non-null  float64
 5   date      35808 non-null  object 
 6   time      35808 non-null  float64
dtypes: float64(5), object(2)
memory usage: 1.9+ MB


In [ ]:
songsan_weather_utc.describe()

,ta,hm,ws,rn,time
count,35808.000000,35808.000000,35808.000000,35808.000000,35808.000000
mean,12.430200,73.253072,1.832423,0.092326,690.000000
std,10.472895,19.031204,1.372122,0.764020,415.336993
min,-50.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,3.500000,60.000000,0.800000,0.000000,345.000000
50%,12.800000,77.000000,1.500000,0.000000,690.000000
75%,21.299999,90.000000,2.500000,0.000000,1035.000000
max,36.700001,252.000000,10.500000,48.099998,1380.000000


In [ ]:
# 모든 날짜 UTC+9
from datetime import timedelta
songsan_weather["mea_ddhr"] = songsan_weather["mea_ddhr"].apply(lambda x: x+timedelta(hours=9))
songsan_weather

,mea_ddhr,ta,hm,ws,rn,date,time
0,2016-03-01 09:00:00,-2.9,63.0,1.0,0.0,2016-03-01,0.0
1,2016-03-01 10:00:00,-1.4,57.0,0.4,0.0,2016-03-01,60.0
2,2016-03-01 11:00:00,-0.2,53.0,0.8,0.0,2016-03-01,120.0
3,2016-03-01 12:00:00,0.6,50.0,1.1,0.0,2016-03-01,180.0
4,2016-03-01 13:00:00,1.6,48.0,1.2,0.0,2016-03-01,240.0
...,...,...,...,...,...,...,...
35803,2020-04-01 04:00:00,8.5,58.0,1.4,0.0,2020-03-31,1140.0
35804,2020-04-01 05:00:00,8.3,56.0,1.1,0.0,2020-03-31,1200.0
35805,2020-04-01 06:00:00,8.4,51.0,1.3,0.0,2020-03-31,1260.0
35806,2020-04-01 07:00:00,9.4,47.0,1.7,0.0,2020-03-31,1320.0


In [ ]:
songsan_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35808 entries, 0 to 35807
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   mea_ddhr  35808 non-null  datetime64[ns]
 1   ta        35808 non-null  float64       
 2   hm        35808 non-null  float64       
 3   ws        35808 non-null  float64       
 4   rn        35808 non-null  float64       
 5   date      35808 non-null  object        
 6   time      35808 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 1.9+ MB


In [ ]:
songsan_weather.describe()

,ta,hm,ws,rn,time
count,35808.000000,35808.000000,35808.000000,35808.000000,35808.000000
mean,12.430200,73.253072,1.832423,0.092326,690.000000
std,10.472895,19.031204,1.372122,0.764020,415.336993
min,-50.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,3.500000,60.000000,0.800000,0.000000,345.000000
50%,12.800000,77.000000,1.500000,0.000000,690.000000
75%,21.299999,90.000000,2.500000,0.000000,1035.000000
max,36.700001,252.000000,10.500000,48.099998,1380.000000


In [ ]:
songsan_weather.name = "songsan_weather"

#### 5. 송악 동네 기록 (1시간)

In [ ]:
# 송악 2016 3월 동네 기록 데이터
songak_2016_03_ta = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2016_03_ta_songak.csv"), encoding = 'ISO-8859-1')
songak_2016_03_hm = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2016_03_hm_songak.csv"), encoding = 'ISO-8859-1')
songak_2016_03_ws = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2016_03_ws_songak.csv"), encoding = 'ISO-8859-1')
songak_2016_03_rn = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2016_03_rn_songak.csv"), encoding = 'ISO-8859-1')

# 송악 2016 동네 기록 데이터
songak_2016_ta = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2016_ta_songak.csv"), encoding = 'ISO-8859-1')
songak_2016_hm = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2016_hm_songak.csv"), encoding = 'ISO-8859-1')
songak_2016_ws = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2016_ws_songak.csv"), encoding = 'ISO-8859-1')
songak_2016_rn = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2016_rn_songak.csv"), encoding = 'ISO-8859-1')

# 송악 2017 동네 기록 데이터
songak_2017_ta = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2017_ta_songak.csv"), encoding = 'ISO-8859-1')
songak_2017_hm = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2017_hm_songak.csv"), encoding = 'ISO-8859-1')
songak_2017_ws = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2017_ws_songak.csv"), encoding = 'ISO-8859-1')
songak_2017_rn = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2017_rn_songak.csv"), encoding = 'ISO-8859-1')

# 송악 2018 동네 기록 데이터
songak_2018_ta = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2018_ta_songak.csv"), encoding = 'ISO-8859-1')
songak_2018_hm = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2018_hm_songak.csv"), encoding = 'ISO-8859-1')
songak_2018_ws = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2018_ws_songak.csv"), encoding = 'ISO-8859-1')
songak_2018_rn = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2018_rn_songak.csv"), encoding = 'ISO-8859-1')

# 송악 2019 동네 기록 데이터
songak_2019_ta = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2019_ta_songak.csv"), encoding = 'ISO-8859-1')
songak_2019_hm = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2019_hm_songak.csv"), encoding = 'ISO-8859-1')
songak_2019_ws = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2019_ws_songak.csv"), encoding = 'ISO-8859-1')
songak_2019_rn = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2019_rn_songak.csv"), encoding = 'ISO-8859-1')

# 송악 2020 동네 기록 데이터
songak_2020_ta = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2020_ta_songak.csv"), encoding = 'ISO-8859-1')
songak_2020_hm = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2020_hm_songak.csv"), encoding = 'ISO-8859-1')
songak_2020_ws = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2020_ws_songak.csv"), encoding = 'ISO-8859-1')
songak_2020_rn = pd.read_csv(os.path.join(DATA_EXPORT_PATH, "songak_local/2020_rn_songak.csv"), encoding = 'ISO-8859-1')


In [ ]:
# 변수명 바꿔주기
songak_2016_03_ta.columns = ["day", "hour", "value"]
songak_2016_03_hm.columns = ["day", "hour", "value"]
songak_2016_03_ws.columns = ["day", "hour", "value"]
songak_2016_03_rn.columns = ["day", "hour", "value"]

songak_2016_ta.columns = ["day", "hour", "value"]
songak_2016_hm.columns = ["day", "hour", "value"]
songak_2016_ws.columns = ["day", "hour", "value"]
songak_2016_rn.columns = ["day", "hour", "value"]

songak_2017_ta.columns = ["day", "hour", "value"]
songak_2017_hm.columns = ["day", "hour", "value"]
songak_2017_ws.columns = ["day", "hour", "value"]
songak_2017_rn.columns = ["day", "hour", "value"]

songak_2018_ta.columns = ["day", "hour", "value"]
songak_2018_hm.columns = ["day", "hour", "value"]
songak_2018_ws.columns = ["day", "hour", "value"]
songak_2018_rn.columns = ["day", "hour", "value"]

songak_2019_ta.columns = ["day", "hour", "value"]
songak_2019_hm.columns = ["day", "hour", "value"]
songak_2019_ws.columns = ["day", "hour", "value"]
songak_2019_rn.columns = ["day", "hour", "value"]

songak_2020_ta.columns = ["day", "hour", "value"]
songak_2020_hm.columns = ["day", "hour", "value"]
songak_2020_ws.columns = ["day", "hour", "value"]
songak_2020_rn.columns = ["day", "hour", "value"]

In [ ]:
songak_2016_03_ta

,day,hour,value
0,1,0.0,-3.4
1,1,100.0,-1.7
2,1,200.0,-0.5
3,1,300.0,0.4
4,1,400.0,1.5
...,...,...,...
740,31,2000.0,4.5
741,31,2100.0,5.2
742,31,2200.0,6.2
743,31,2300.0,9.8


In [ ]:
songak_2016_ta

,day,hour,value
0,1,0.0,13.800000
1,1,100.0,16.400000
2,1,200.0,18.700001
3,1,300.0,20.200001
4,1,400.0,21.500000
...,...,...,...
6604,31,2000.0,0.300000
6605,31,2100.0,0.600000
6606,31,2200.0,0.600000
6607,31,2300.0,0.300000


In [ ]:
# 마지막 row 결측치 제거
songak_2016_03_ta = songak_2016_03_ta.drop(songak_2016_03_ta.shape[0]-1)
songak_2016_03_hm = songak_2016_03_hm.drop(songak_2016_03_hm.shape[0]-1)
songak_2016_03_ws = songak_2016_03_ws.drop(songak_2016_03_ws.shape[0]-1)
songak_2016_03_rn = songak_2016_03_rn.drop(songak_2016_03_rn.shape[0]-1)

songak_2016_ta = songak_2016_ta.drop(len(songak_2016_ta)-1)
songak_2016_hm = songak_2016_hm.drop(len(songak_2016_hm)-1)
songak_2016_ws = songak_2016_ws.drop(len(songak_2016_ws)-1)
songak_2016_rn = songak_2016_rn.drop(len(songak_2016_rn)-1)

songak_2017_ta = songak_2017_ta.drop(len(songak_2017_ta)-1)
songak_2017_hm = songak_2017_hm.drop(len(songak_2017_hm)-1)
songak_2017_ws = songak_2017_ws.drop(len(songak_2017_ws)-1)
songak_2017_rn = songak_2017_rn.drop(len(songak_2017_rn)-1)

songak_2018_ta = songak_2018_ta.drop(len(songak_2018_ta)-1)
songak_2018_hm = songak_2018_hm.drop(len(songak_2018_hm)-1)
songak_2018_ws = songak_2018_ws.drop(len(songak_2018_ws)-1)
songak_2018_rn = songak_2018_rn.drop(len(songak_2018_rn)-1)

songak_2019_ta = songak_2019_ta.drop(len(songak_2019_ta)-1)
songak_2019_hm = songak_2019_hm.drop(len(songak_2019_hm)-1)
songak_2019_ws = songak_2019_ws.drop(len(songak_2019_ws)-1)
songak_2019_rn = songak_2019_rn.drop(len(songak_2019_rn)-1)

songak_2020_ta = songak_2020_ta.drop(len(songak_2020_ta)-1)
songak_2020_hm = songak_2020_hm.drop(len(songak_2020_hm)-1)
songak_2020_ws = songak_2020_ws.drop(len(songak_2020_ws)-1)
songak_2020_rn = songak_2020_rn.drop(len(songak_2020_rn)-1)

In [ ]:
songak_2016_ta

,day,hour,value
0,1,0.0,13.800000
1,1,100.0,16.400000
2,1,200.0,18.700001
3,1,300.0,20.200001
4,1,400.0,21.500000
...,...,...,...
6603,31,1900.0,0.900000
6604,31,2000.0,0.300000
6605,31,2100.0,0.600000
6606,31,2200.0,0.600000


In [ ]:
# 송산 기존 데이터 첫행 데이터 추가
# Start : 20160401
data = {'day':["Start : 20160401"], 'hour': [0], 'value': [0]}
temp = pd.DataFrame(data)
temp

,day,hour,value
0,Start : 20160401,0,0


In [ ]:
# 송산 2016년도 병합
songak_2016_ta = pd.concat([songak_2016_03_ta, temp, songak_2016_ta], ignore_index=True)
songak_2016_hm = pd.concat([songak_2016_03_hm, temp, songak_2016_hm], ignore_index=True)
songak_2016_rn = pd.concat([songak_2016_03_rn, temp, songak_2016_rn], ignore_index=True)
songak_2016_ws = pd.concat([songak_2016_03_ws, temp, songak_2016_ws], ignore_index=True)
songak_2016_ta

,day,hour,value
0,1,0.0,-3.4
1,1,100.0,-1.7
2,1,200.0,-0.5
3,1,300.0,0.4
4,1,400.0,1.5
...,...,...,...
7348,31,1900.0,0.9
7349,31,2000.0,0.3
7350,31,2100.0,0.6
7351,31,2200.0,0.6


In [ ]:
songak_2016_ta[743:750]

,day,hour,value
743,31,2300.0,9.800000
744,Start : 20160401,0.0,0.000000
745,1,0.0,13.800000
746,1,100.0,16.400000
747,1,200.0,18.700001
748,1,300.0,20.200001
749,1,400.0,21.500000


In [ ]:
# 4개의 정보의 크기가 같은지 확인
print("====== 송악 2016 ======")
print(songak_2016_ta.shape)
print(songak_2016_hm.shape)
print(songak_2016_ws.shape)
print(songak_2016_rn.shape)

print("====== 송악 2017 ======")
print(songak_2017_ta.shape)
print(songak_2017_hm.shape)
print(songak_2017_ws.shape)
print(songak_2017_rn.shape)

print("====== 송악 2018 ======")
print(songak_2018_ta.shape)
print(songak_2018_hm.shape)
print(songak_2018_ws.shape)
print(songak_2018_rn.shape)

print("====== 송악 2019 ======")
print(songak_2019_ta.shape)
print(songak_2019_hm.shape)
print(songak_2019_ws.shape)
print(songak_2019_rn.shape)

print("====== 송악 2020 ======")
print(songak_2020_ta.shape)
print(songak_2020_hm.shape)
print(songak_2020_ws.shape)
print(songak_2020_rn.shape)

====== 송악 2016 ======
(7353, 3)
(7353, 3)
(7353, 3)
(7353, 3)
====== 송악 2017 ======
(8771, 3)
(8771, 3)
(8771, 3)
(8771, 3)
====== 송악 2018 ======
(8771, 3)
(8771, 3)
(8771, 3)
(8771, 3)
====== 송악 2019 ======
(8771, 3)
(8771, 3)
(8771, 3)
(8771, 3)
====== 송악 2020 ======
(2186, 3)
(2186, 3)
(2186, 3)
(2186, 3)


In [ ]:
# 송악 2016 병합
import datetime
date_start = datetime.datetime(2016, 3, 1, 0, 0, 0)
date_end = datetime.datetime(2016, 12, 31, 23, 0, 0)
dataList = [songak_2016_ta, songak_2016_hm, songak_2016_ws, songak_2016_rn]
songak_2016 = localWeatherToDf(date_start, date_end, dataList)
songak_2016

2016-03-01 00:00:00 작업중 ...
2016-03-21 19:00:00 작업중 ...
2016-04-11 14:00:00 작업중 ...
2016-05-02 09:00:00 작업중 ...
2016-05-23 05:00:00 작업중 ...
2016-06-13 00:00:00 작업중 ...
2016-07-03 19:00:00 작업중 ...
2016-07-24 15:00:00 작업중 ...
2016-08-14 10:00:00 작업중 ...
2016-09-04 05:00:00 작업중 ...
2016-09-25 01:00:00 작업중 ...
2016-10-15 20:00:00 작업중 ...
2016-11-05 15:00:00 작업중 ...
2016-11-26 11:00:00 작업중 ...
2016-12-17 06:00:00 작업중 ...


,mea_ddhr,ta,hm,ws,rn
0,2016-03-01 00:00:00,-3.4,62.0,1.3,0.0
1,2016-03-01 01:00:00,-1.7,57.0,0.5,0.0
2,2016-03-01 02:00:00,-0.5,52.0,0.7,0.0
3,2016-03-01 03:00:00,0.4,50.0,1.1,0.0
4,2016-03-01 04:00:00,1.5,47.0,1.0,0.0
...,...,...,...,...,...
7339,2016-12-31 19:00:00,0.9,94.0,0.4,0.0
7340,2016-12-31 20:00:00,0.3,94.0,0.2,0.0
7341,2016-12-31 21:00:00,0.6,92.0,0.2,0.0
7342,2016-12-31 22:00:00,0.6,91.0,0.6,0.0


In [ ]:
# 송악 2017 병합
import datetime
date_start = datetime.datetime(2017, 1, 1, 0, 0, 0)
date_end = datetime.datetime(2017, 12, 31, 23, 0, 0)
dataList = [songak_2017_ta, songak_2017_hm, songak_2017_ws, songak_2017_rn]
songak_2017 = localWeatherToDf(date_start, date_end, dataList)
songak_2017

2017-01-01 00:00:00 작업중 ...
2017-01-21 19:00:00 작업중 ...
2017-02-11 14:00:00 작업중 ...
2017-03-04 09:00:00 작업중 ...
2017-03-25 05:00:00 작업중 ...
2017-04-15 00:00:00 작업중 ...
2017-05-05 19:00:00 작업중 ...
2017-05-26 15:00:00 작업중 ...
2017-06-16 10:00:00 작업중 ...
2017-07-07 05:00:00 작업중 ...
2017-07-28 01:00:00 작업중 ...
2017-08-17 20:00:00 작업중 ...
2017-09-07 15:00:00 작업중 ...
2017-09-28 11:00:00 작업중 ...
2017-10-19 06:00:00 작업중 ...
2017-11-09 01:00:00 작업중 ...
2017-11-29 21:00:00 작업중 ...
2017-12-20 16:00:00 작업중 ...


,mea_ddhr,ta,hm,ws,rn
0,2017-01-01 00:00:00,1.6,90.0,1.6,0.0
1,2017-01-01 01:00:00,3.4,86.0,0.6,0.0
2,2017-01-01 02:00:00,4.8,80.0,0.7,0.0
3,2017-01-01 03:00:00,6.5,71.0,1.5,0.0
4,2017-01-01 04:00:00,8.0,67.0,2.3,0.0
...,...,...,...,...,...
8755,2017-12-31 19:00:00,-5.2,79.0,1.1,0.0
8756,2017-12-31 20:00:00,-5.5,75.0,1.4,0.0
8757,2017-12-31 21:00:00,-5.2,75.0,1.7,0.0
8758,2017-12-31 22:00:00,-5.0,74.0,1.3,0.0


In [ ]:
# 송악 2018 병합
import datetime
date_start = datetime.datetime(2018, 1, 1, 0, 0, 0)
date_end = datetime.datetime(2018, 12, 31, 23, 0, 0)
dataList = [songak_2018_ta, songak_2018_hm, songak_2018_ws, songak_2018_rn]
songak_2018 = localWeatherToDf(date_start, date_end, dataList)
songak_2018

2018-01-01 00:00:00 작업중 ...
2018-01-21 19:00:00 작업중 ...
2018-02-11 14:00:00 작업중 ...
2018-03-04 09:00:00 작업중 ...
2018-03-25 05:00:00 작업중 ...
2018-04-15 00:00:00 작업중 ...
2018-05-05 19:00:00 작업중 ...
2018-05-26 15:00:00 작업중 ...
2018-06-16 10:00:00 작업중 ...
2018-07-07 05:00:00 작업중 ...
2018-07-28 01:00:00 작업중 ...
2018-08-17 20:00:00 작업중 ...
2018-09-07 15:00:00 작업중 ...
2018-09-28 11:00:00 작업중 ...
2018-10-19 06:00:00 작업중 ...
2018-11-09 01:00:00 작업중 ...
2018-11-29 21:00:00 작업중 ...
2018-12-20 16:00:00 작업중 ...


,mea_ddhr,ta,hm,ws,rn
0,2018-01-01 00:00:00,1.0,51.0,1.0,0.0
1,2018-01-01 01:00:00,2.1,40.0,1.1,0.0
2,2018-01-01 02:00:00,3.3,30.0,0.7,0.0
3,2018-01-01 03:00:00,3.5,30.0,0.9,0.0
4,2018-01-01 04:00:00,3.0,31.0,0.8,0.0
...,...,...,...,...,...
8755,2018-12-31 19:00:00,-3.7,74.0,1.1,0.0
8756,2018-12-31 20:00:00,-5.6,70.0,0.2,0.0
8757,2018-12-31 21:00:00,-5.5,78.0,0.3,0.0
8758,2018-12-31 22:00:00,-4.8,83.0,0.1,0.0


In [ ]:
# 송악 2019 병합
import datetime
date_start = datetime.datetime(2019, 1, 1, 0, 0, 0)
date_end = datetime.datetime(2019, 12, 31, 23, 0, 0)
dataList = [songak_2019_ta, songak_2019_hm, songak_2019_ws, songak_2019_rn]
songak_2019 = localWeatherToDf(date_start, date_end, dataList)
songak_2019

2019-01-01 00:00:00 작업중 ...
2019-01-21 19:00:00 작업중 ...
2019-02-11 14:00:00 작업중 ...
2019-03-04 09:00:00 작업중 ...
2019-03-25 05:00:00 작업중 ...
2019-04-15 00:00:00 작업중 ...
2019-05-05 19:00:00 작업중 ...
2019-05-26 15:00:00 작업중 ...
2019-06-16 10:00:00 작업중 ...
2019-07-07 05:00:00 작업중 ...
2019-07-28 01:00:00 작업중 ...
2019-08-17 20:00:00 작업중 ...
2019-09-07 15:00:00 작업중 ...
2019-09-28 11:00:00 작업중 ...
2019-10-19 06:00:00 작업중 ...
2019-11-09 01:00:00 작업중 ...
2019-11-29 21:00:00 작업중 ...
2019-12-20 16:00:00 작업중 ...


,mea_ddhr,ta,hm,ws,rn
0,2019-01-01 00:00:00,-3.0,70.0,0.6,0.0
1,2019-01-01 01:00:00,-2.0,71.0,1.3,0.0
2,2019-01-01 02:00:00,-0.7,70.0,1.4,0.0
3,2019-01-01 03:00:00,-1.1,80.0,1.6,0.0
4,2019-01-01 04:00:00,-2.1,91.0,1.3,0.0
...,...,...,...,...,...
8755,2019-12-31 19:00:00,-4.5,86.0,0.7,0.0
8756,2019-12-31 20:00:00,-4.1,92.0,1.0,0.0
8757,2019-12-31 21:00:00,-3.2,94.0,0.4,0.0
8758,2019-12-31 22:00:00,-3.0,91.0,0.3,0.0


In [ ]:
# 송악 2020 병합
import datetime
date_start = datetime.datetime(2020, 1, 1, 0, 0, 0)
date_end = datetime.datetime(2020, 3, 31, 23, 0, 0)
dataList = [songak_2020_ta, songak_2020_hm, songak_2020_ws, songak_2020_rn]
songak_2020 = localWeatherToDf(date_start, date_end, dataList)
songak_2020

2020-01-01 00:00:00 작업중 ...
2020-01-21 19:00:00 작업중 ...
2020-02-11 14:00:00 작업중 ...
2020-03-03 09:00:00 작업중 ...
2020-03-24 05:00:00 작업중 ...


,mea_ddhr,ta,hm,ws,rn
0,2020-01-01 00:00:00,-0.2,81.0,1.1,0.0
1,2020-01-01 01:00:00,2.4,64.0,0.7,0.0
2,2020-01-01 02:00:00,2.0,69.0,0.8,0.0
3,2020-01-01 03:00:00,1.2,72.0,0.7,0.0
4,2020-01-01 04:00:00,1.3,72.0,0.6,0.0
...,...,...,...,...,...
2179,2020-03-31 19:00:00,8.5,58.0,1.4,0.0
2180,2020-03-31 20:00:00,8.3,56.0,1.1,0.0
2181,2020-03-31 21:00:00,8.4,51.0,1.3,0.0
2182,2020-03-31 22:00:00,9.4,47.0,1.7,0.0


In [ ]:
# 송악 모든년도 병합
songak_weather = pd.concat([songak_2016, songak_2017, songak_2018, songak_2019, songak_2020], ignore_index=True)
songak_weather

,mea_ddhr,ta,hm,ws,rn
0,2016-03-01 00:00:00,-3.4,62.0,1.3,0.0
1,2016-03-01 01:00:00,-1.7,57.0,0.5,0.0
2,2016-03-01 02:00:00,-0.5,52.0,0.7,0.0
3,2016-03-01 03:00:00,0.4,50.0,1.1,0.0
4,2016-03-01 04:00:00,1.5,47.0,1.0,0.0
...,...,...,...,...,...
35803,2020-03-31 19:00:00,8.5,58.0,1.4,0.0
35804,2020-03-31 20:00:00,8.3,56.0,1.1,0.0
35805,2020-03-31 21:00:00,8.4,51.0,1.3,0.0
35806,2020-03-31 22:00:00,9.4,47.0,1.7,0.0


In [ ]:
# date, time 변수 제작
# utc 기준으로 된 data 제작
import datetime
songak_weather_utc = createDateTime(songak_weather)
songak_weather_utc

,mea_ddhr,ta,hm,ws,rn,date,time
0,2016-03-01 00:00:00,-3.4,62.0,1.3,0.0,2016-03-01,0.0
1,2016-03-01 01:00:00,-1.7,57.0,0.5,0.0,2016-03-01,60.0
2,2016-03-01 02:00:00,-0.5,52.0,0.7,0.0,2016-03-01,120.0
3,2016-03-01 03:00:00,0.4,50.0,1.1,0.0,2016-03-01,180.0
4,2016-03-01 04:00:00,1.5,47.0,1.0,0.0,2016-03-01,240.0
...,...,...,...,...,...,...,...
35803,2020-03-31 19:00:00,8.5,58.0,1.4,0.0,2020-03-31,1140.0
35804,2020-03-31 20:00:00,8.3,56.0,1.1,0.0,2020-03-31,1200.0
35805,2020-03-31 21:00:00,8.4,51.0,1.3,0.0,2020-03-31,1260.0
35806,2020-03-31 22:00:00,9.4,47.0,1.7,0.0,2020-03-31,1320.0


In [ ]:
songak_weather_utc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35808 entries, 0 to 35807
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   mea_ddhr  35808 non-null  object 
 1   ta        35808 non-null  float64
 2   hm        35808 non-null  float64
 3   ws        35808 non-null  float64
 4   rn        35808 non-null  float64
 5   date      35808 non-null  object 
 6   time      35808 non-null  float64
dtypes: float64(5), object(2)
memory usage: 1.9+ MB


In [ ]:
songak_weather_utc.describe()

,ta,hm,ws,rn,time
count,35808.000000,35808.000000,35808.000000,35808.000000,35808.000000
mean,12.399447,72.192583,1.761277,0.097970,690.000000
std,10.509478,19.043617,1.321793,0.767751,415.336993
min,-50.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,3.400000,59.000000,0.800000,0.000000,345.000000
50%,12.800000,76.000000,1.500000,0.000000,690.000000
75%,21.299999,89.000000,2.400000,0.000000,1035.000000
max,36.700001,100.000000,10.800000,34.599998,1380.000000


In [ ]:
# date, time 변수 제작
# 한국시간 기준으로 된 data 제작
import datetime
songak_weather = createDateTime(songak_weather)

In [ ]:
# 모든 날짜 UTC+9
from datetime import timedelta
songak_weather["mea_ddhr"] = songak_weather["mea_ddhr"].apply(lambda x: x+timedelta(hours=9))
songak_weather

,mea_ddhr,ta,hm,ws,rn,date,time
0,2016-03-01 09:00:00,-3.4,62.0,1.3,0.0,2016-03-01,0.0
1,2016-03-01 10:00:00,-1.7,57.0,0.5,0.0,2016-03-01,60.0
2,2016-03-01 11:00:00,-0.5,52.0,0.7,0.0,2016-03-01,120.0
3,2016-03-01 12:00:00,0.4,50.0,1.1,0.0,2016-03-01,180.0
4,2016-03-01 13:00:00,1.5,47.0,1.0,0.0,2016-03-01,240.0
...,...,...,...,...,...,...,...
35803,2020-04-01 04:00:00,8.5,58.0,1.4,0.0,2020-03-31,1140.0
35804,2020-04-01 05:00:00,8.3,56.0,1.1,0.0,2020-03-31,1200.0
35805,2020-04-01 06:00:00,8.4,51.0,1.3,0.0,2020-03-31,1260.0
35806,2020-04-01 07:00:00,9.4,47.0,1.7,0.0,2020-03-31,1320.0


In [ ]:
songak_weather

,mea_ddhr,ta,hm,ws,rn,date,time
0,2016-03-01 09:00:00,-3.4,62.0,1.3,0.0,2016-03-01,0.0
1,2016-03-01 10:00:00,-1.7,57.0,0.5,0.0,2016-03-01,60.0
2,2016-03-01 11:00:00,-0.5,52.0,0.7,0.0,2016-03-01,120.0
3,2016-03-01 12:00:00,0.4,50.0,1.1,0.0,2016-03-01,180.0
4,2016-03-01 13:00:00,1.5,47.0,1.0,0.0,2016-03-01,240.0
...,...,...,...,...,...,...,...
35803,2020-04-01 04:00:00,8.5,58.0,1.4,0.0,2020-03-31,1140.0
35804,2020-04-01 05:00:00,8.3,56.0,1.1,0.0,2020-03-31,1200.0
35805,2020-04-01 06:00:00,8.4,51.0,1.3,0.0,2020-03-31,1260.0
35806,2020-04-01 07:00:00,9.4,47.0,1.7,0.0,2020-03-31,1320.0


In [ ]:
songak_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35808 entries, 0 to 35807
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   mea_ddhr  35808 non-null  datetime64[ns]
 1   ta        35808 non-null  float64       
 2   hm        35808 non-null  float64       
 3   ws        35808 non-null  float64       
 4   rn        35808 non-null  float64       
 5   date      35808 non-null  object        
 6   time      35808 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 1.9+ MB


In [ ]:
songak_weather.name = "songak_weather"

### 3. 결측치 채우기

##### 0. 사용하는 함수

In [ ]:
import datetime
import copy
def fillMissingValue(main_df, replace_df, col, threshold):
  ret_df = copy.deepcopy(main_df)
  # ret_df.name = main_df.name
  rep_df = copy.deepcopy(replace_df)
  ret_df["date"] = ret_df["date"].apply(lambda x: str(x))
  rep_df["date"] = rep_df["date"].apply(lambda x: str(x))

  nan_df = ret_df[ret_df[col].isnull()]
  # nan_df["date"] = nan_df["date"].apply(lambda x: str(x))

  grouped = nan_df.groupby(nan_df['date']).size()
  nanDateList = grouped.index
  nanDateValues = grouped.values


  for i in range(len(nanDateList)):
    if nanDateValues[i] >= threshold:
      a = ret_df[ret_df["date"] == nanDateList[i]]
      b = rep_df[rep_df["date"] == nanDateList[i]]
      
      if a.shape[0] == b.shape[0]:
        for idx1, idx2 in zip(a.index, b.index):
          if a.loc[idx1, col] != a.loc[idx1, col]:
            
            # print(a.loc[idx1, col])
            a.loc[idx1, col] = b.loc[idx2, col]
            # print(a.loc[idx1, col])

        ret_df.loc[a.index, col] = a[col]
  return ret_df




In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ta가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
import datetime
def getNanGroup(df, col):
  a = df[df[col].isnull()]
  a["date"] = a["date"].apply(lambda x: str(x))
  return a.groupby(a['date']).size()



In [ ]:
def compareColumn(main_df, comp_df, col, date):
  a = copy.deepcopy(main_df)
  b = copy.deepcopy(comp_df)
  # a.name = main_df.name+"_"
  # b.name = comp_df.name+"_"
  a["date"] = a["date"].apply(lambda x: str(x))
  b["date"] = b["date"].apply(lambda x: str(x))
  a = a[a["date"] == date]
  b = b[b["date"] == date]

  
  if a.shape[0] == b.shape[0]: 
    c = pd.merge(a, b, on="mea_ddhr")
    c = c[["mea_ddhr", f'{col}_x', f'{col}_y']]
    c.columns = ["mea_ddhr", f'left_{col}', f'right_{col}']
  
    return c
  else:
    return -1


#### 1. 당진 616

##### 1. ta

In [ ]:
# 전체 nan 확인
is_NaN = dangjin_weather.isnull()
row_has_NaN = is_NaN.any(axis=1)
dangjin_weather[row_has_NaN]

,mea_ddhr,ta,hm,ws,rn,date,time
1003,2016-05-12 19:00:00,17.3,79.1,NaN,0.0,2016-05-12,1140.0
1193,2016-05-20 17:00:00,25.3,52.1,NaN,0.0,2016-05-20,1020.0
2865,2016-07-29 11:00:00,NaN,NaN,NaN,0.5,2016-07-29,660.0
3144,2016-08-10 23:00:00,23.9,92.3,NaN,0.0,2016-08-10,1380.0
3145,2016-08-11 00:00:00,23.8,92.3,NaN,0.0,2016-08-11,0.0
...,...,...,...,...,...,...,...
30229,2019-09-18 15:00:00,22.0,98.9,NaN,0.0,2019-09-18,900.0
30372,2019-09-24 14:00:00,NaN,NaN,NaN,0.0,2019-09-24,840.0
30392,2019-09-25 10:00:00,21.1,78.2,NaN,0.0,2019-09-25,600.0
30513,2019-09-30 11:00:00,22.0,73.3,NaN,0.0,2019-09-30,660.0


In [ ]:
getNanGroup(dangjin_weather, "ta")

date
2016-07-29     1
2016-08-12     1
2016-08-20     1
2016-09-08     1
2016-10-06     1
2016-10-17     1
2016-12-02     3
2016-12-06     2
2016-12-07     2
2016-12-10     3
2016-12-11     8
2016-12-12    11
2016-12-15     1
2016-12-16    12
2017-07-09     1
2017-07-24     1
2017-07-31     2
2017-08-01     1
2017-08-02     2
2017-09-26     1
2017-10-11     1
2017-10-13     1
2018-06-21     1
2019-04-19     1
2019-08-03     1
2019-08-17     1
2019-08-22     1
2019-08-29     5
2019-09-05     1
2019-09-24     1
2019-11-17     1
dtype: int64

In [ ]:
# 역으로 하나하나 날짜에 대해 추적해본다.
import datetime
# sinpyeong_weather[sinpyeong_weather["date"] == datetime.datetime.strptime("2016-12-16", "%Y-%m-%d").date()]
dangjin_weather[dangjin_weather["date"] == datetime.datetime.strptime("2016-12-16", "%Y-%m-%d").date()]

,mea_ddhr,ta,hm,ws,rn,date,time
6183,2016-12-16 00:00:00,NaN,61.9,1.4,0.0,2016-12-16,0.0
6184,2016-12-16 01:00:00,NaN,62.8,1.5,0.0,2016-12-16,60.0
6185,2016-12-16 02:00:00,NaN,58.7,1.6,0.0,2016-12-16,120.0
6186,2016-12-16 03:00:00,NaN,66.1,0.4,0.0,2016-12-16,180.0
6187,2016-12-16 04:00:00,NaN,69.0,0.4,0.0,2016-12-16,240.0
6188,2016-12-16 05:00:00,NaN,66.3,0.8,0.0,2016-12-16,300.0
6189,2016-12-16 06:00:00,NaN,73.6,0.2,0.0,2016-12-16,360.0
6190,2016-12-16 07:00:00,NaN,76.3,0.1,0.0,2016-12-16,420.0
6191,2016-12-16 08:00:00,NaN,78.9,0.8,0.0,2016-12-16,480.0
6192,2016-12-16 09:00:00,NaN,72.7,0.1,0.0,2016-12-16,540.0


In [ ]:
compareColumn(dangjin_weather, sinpyeong_weather, "ta", "2016-12-16")

,mea_ddhr,left_ta,right_ta
0,2016-12-16 00:00:00,NaN,-4.8
1,2016-12-16 01:00:00,NaN,-5.1
2,2016-12-16 02:00:00,NaN,-5.2
3,2016-12-16 03:00:00,NaN,-6.0
4,2016-12-16 04:00:00,NaN,-7.6
5,2016-12-16 05:00:00,NaN,-6.6
6,2016-12-16 06:00:00,NaN,-7.5
7,2016-12-16 07:00:00,NaN,-8.8
8,2016-12-16 08:00:00,NaN,-8.7
9,2016-12-16 09:00:00,NaN,-6.5


In [ ]:
# 결측치가 5개 이상인 것에 대해 대체한다.
dangjin_weather = fillMissingValue(dangjin_weather, sinpyeong_weather, "ta", 5)

In [ ]:
compareColumn(dangjin_weather, sinpyeong_weather, "ta", "2016-12-16")

,mea_ddhr,left_ta,right_ta
0,2016-12-16 00:00:00,-4.8,-4.8
1,2016-12-16 01:00:00,-5.1,-5.1
2,2016-12-16 02:00:00,-5.2,-5.2
3,2016-12-16 03:00:00,-6.0,-6.0
4,2016-12-16 04:00:00,-7.6,-7.6
5,2016-12-16 05:00:00,-6.6,-6.6
6,2016-12-16 06:00:00,-7.5,-7.5
7,2016-12-16 07:00:00,-8.8,-8.8
8,2016-12-16 08:00:00,-8.7,-8.7
9,2016-12-16 09:00:00,-6.5,-6.5


In [ ]:
# 사라졌는지 확인
getNanGroup(dangjin_weather, "ta")

date
2016-07-29    1
2016-08-12    1
2016-08-20    1
2016-09-08    1
2016-10-06    1
2016-10-17    1
2016-12-02    3
2016-12-06    2
2016-12-07    2
2016-12-10    3
2016-12-15    1
2017-07-09    1
2017-07-24    1
2017-07-31    2
2017-08-01    1
2017-08-02    2
2017-09-26    1
2017-10-11    1
2017-10-13    1
2018-06-21    1
2019-04-19    1
2019-08-03    1
2019-08-17    1
2019-08-22    1
2019-09-05    1
2019-09-24    1
2019-11-17    1
dtype: int64

##### 2. hm

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, hm가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(dangjin_weather, "hm")

date
2016-07-29    1
2016-08-12    1
2016-08-20    1
2016-09-08    1
2016-10-06    1
2016-10-17    1
2016-12-12    1
2016-12-16    1
2017-07-09    1
2017-07-24    1
2017-07-31    2
2017-08-01    1
2017-08-02    2
2017-09-26    1
2017-10-11    1
2017-10-13    1
2018-06-21    1
2019-08-03    1
2019-08-17    1
2019-08-22    1
2019-08-29    5
2019-09-05    1
2019-09-24    1
2019-11-17    1
dtype: int64

In [ ]:
# 결측치가 5개 이상인 것에 대해 대체한다.
dangjin_weather = fillMissingValue(dangjin_weather, seosan_weather, "hm", 5)

In [ ]:
# 확인
getNanGroup(dangjin_weather, "hm")

date
2016-07-29    1
2016-08-12    1
2016-08-20    1
2016-09-08    1
2016-10-06    1
2016-10-17    1
2016-12-12    1
2016-12-16    1
2017-07-09    1
2017-07-24    1
2017-07-31    2
2017-08-01    1
2017-08-02    2
2017-09-26    1
2017-10-11    1
2017-10-13    1
2018-06-21    1
2019-08-03    1
2019-08-17    1
2019-08-22    1
2019-09-05    1
2019-09-24    1
2019-11-17    1
dtype: int64

##### 3. ws

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(dangjin_weather, "ws")

date
2016-05-12     1
2016-05-20     1
2016-07-29     1
2016-08-10     1
2016-08-11    12
2016-08-12     1
2016-08-20     1
2016-09-08     2
2016-10-06     1
2016-10-17     2
2016-12-12     1
2016-12-16     2
2017-01-05     2
2017-07-09     2
2017-07-20     1
2017-07-24     1
2017-07-31     2
2017-08-01     1
2017-08-02     2
2017-08-06     1
2017-09-20     1
2017-09-26     1
2017-09-27     1
2017-10-11     2
2017-10-13     1
2018-06-09     1
2018-06-21     1
2018-12-17     2
2018-12-27     1
2019-05-22     1
2019-07-23     1
2019-08-01     1
2019-08-03     1
2019-08-17     1
2019-08-20     1
2019-08-22     1
2019-08-29     5
2019-09-05     1
2019-09-10     1
2019-09-18     1
2019-09-24     1
2019-09-25     1
2019-09-30     1
2019-11-17     1
dtype: int64

In [ ]:
# 추적하는 코드
compareColumn(dangjin_weather, sinpyeong_weather, "ws", "2016-08-11")


,mea_ddhr,left_ws,right_ws
0,2016-08-11 00:00:00,NaN,0.8
1,2016-08-11 01:00:00,NaN,1.1
2,2016-08-11 02:00:00,NaN,1.0
3,2016-08-11 03:00:00,NaN,1.1
4,2016-08-11 04:00:00,NaN,1.0
5,2016-08-11 05:00:00,NaN,1.3
6,2016-08-11 06:00:00,NaN,0.9
7,2016-08-11 07:00:00,NaN,0.7
8,2016-08-11 08:00:00,NaN,1.2
9,2016-08-11 09:00:00,NaN,0.6


In [ ]:
# 결측치가 5개 이상인 것에 대해 대체한다.
dangjin_weather = fillMissingValue(dangjin_weather, sinpyeong_weather, "ws", 5)

In [ ]:
# 추적하는 코드
compareColumn(dangjin_weather, sinpyeong_weather, "ws", "2016-08-11")


,mea_ddhr,left_ws,right_ws
0,2016-08-11 00:00:00,0.8,0.8
1,2016-08-11 01:00:00,1.1,1.1
2,2016-08-11 02:00:00,1.0,1.0
3,2016-08-11 03:00:00,1.1,1.1
4,2016-08-11 04:00:00,1.0,1.0
5,2016-08-11 05:00:00,1.3,1.3
6,2016-08-11 06:00:00,0.9,0.9
7,2016-08-11 07:00:00,0.7,0.7
8,2016-08-11 08:00:00,1.2,1.2
9,2016-08-11 09:00:00,0.6,0.6


In [ ]:
# 확인
getNanGroup(dangjin_weather, "ws")

date
2016-05-12    1
2016-05-20    1
2016-07-29    1
2016-08-10    1
2016-08-12    1
2016-08-20    1
2016-09-08    2
2016-10-06    1
2016-10-17    2
2016-12-12    1
2016-12-16    2
2017-01-05    2
2017-07-09    2
2017-07-20    1
2017-07-24    1
2017-07-31    2
2017-08-01    1
2017-08-02    2
2017-08-06    1
2017-09-20    1
2017-09-26    1
2017-09-27    1
2017-10-11    2
2017-10-13    1
2018-06-09    1
2018-06-21    1
2018-12-17    2
2018-12-27    1
2019-05-22    1
2019-07-23    1
2019-08-01    1
2019-08-03    1
2019-08-17    1
2019-08-20    1
2019-08-22    1
2019-09-05    1
2019-09-10    1
2019-09-18    1
2019-09-24    1
2019-09-25    1
2019-09-30    1
2019-11-17    1
dtype: int64

##### 4. rn

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, rn가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(dangjin_weather, "rn")

date
2017-04-19    13
2017-04-20     1
2017-07-24     3
2017-07-31    10
2017-08-01     1
2017-08-06    13
2017-08-07     1
2017-08-09    12
2018-04-15    22
2018-04-16     1
2018-09-07    15
2018-09-08     1
2018-12-17     1
2019-08-01     5
2019-08-02     1
2019-08-29    12
2019-08-30     1
dtype: int64

In [ ]:
# 역으로 하나하나 날짜에 대해 추적해본다.
compareColumn(dangjin_weather, sinpyeong_weather, "rn", "2017-08-06")

,mea_ddhr,left_rn,right_rn
0,2017-08-06 00:00:00,0.0,0.0
1,2017-08-06 01:00:00,0.0,0.0
2,2017-08-06 02:00:00,0.0,0.0
3,2017-08-06 03:00:00,0.0,0.0
4,2017-08-06 04:00:00,0.0,0.0
5,2017-08-06 05:00:00,0.0,0.0
6,2017-08-06 06:00:00,0.0,0.0
7,2017-08-06 07:00:00,0.0,0.0
8,2017-08-06 08:00:00,0.0,3.5
9,2017-08-06 09:00:00,0.0,23.5


In [ ]:
# 결측치가 5개 이상인 것에 대해 대체한다.
dangjin_weather = fillMissingValue(dangjin_weather, sinpyeong_weather, "rn", 5)

In [ ]:
# 역으로 하나하나 날짜에 대해 추적해본다.
compareColumn(dangjin_weather, sinpyeong_weather, "rn", "2017-08-06")

,mea_ddhr,left_rn,right_rn
0,2017-08-06 00:00:00,0.0,0.0
1,2017-08-06 01:00:00,0.0,0.0
2,2017-08-06 02:00:00,0.0,0.0
3,2017-08-06 03:00:00,0.0,0.0
4,2017-08-06 04:00:00,0.0,0.0
5,2017-08-06 05:00:00,0.0,0.0
6,2017-08-06 06:00:00,0.0,0.0
7,2017-08-06 07:00:00,0.0,0.0
8,2017-08-06 08:00:00,0.0,3.5
9,2017-08-06 09:00:00,0.0,23.5


In [ ]:
# 확인
getNanGroup(dangjin_weather, "ws")

date
2016-05-12    1
2016-05-20    1
2016-07-29    1
2016-08-10    1
2016-08-12    1
2016-08-20    1
2016-09-08    2
2016-10-06    1
2016-10-17    2
2016-12-12    1
2016-12-16    2
2017-01-05    2
2017-07-09    2
2017-07-20    1
2017-07-24    1
2017-07-31    2
2017-08-01    1
2017-08-02    2
2017-08-06    1
2017-09-20    1
2017-09-26    1
2017-09-27    1
2017-10-11    2
2017-10-13    1
2018-06-09    1
2018-06-21    1
2018-12-17    2
2018-12-27    1
2019-05-22    1
2019-07-23    1
2019-08-01    1
2019-08-03    1
2019-08-17    1
2019-08-20    1
2019-08-22    1
2019-09-05    1
2019-09-10    1
2019-09-18    1
2019-09-24    1
2019-09-25    1
2019-09-30    1
2019-11-17    1
dtype: int64

##### 제거 후 확인

In [ ]:
dangjin_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34893 entries, 0 to 34892
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   mea_ddhr  34893 non-null  datetime64[ns]
 1   ta        34858 non-null  float64       
 2   hm        34868 non-null  float64       
 3   ws        34842 non-null  float64       
 4   rn        34872 non-null  float64       
 5   date      34893 non-null  object        
 6   time      34893 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 1.9+ MB


#### 2. 신평 637

In [ ]:
# 전체 nan 확인
is_NaN = sinpyeong_weather.isnull()
row_has_NaN = is_NaN.any(axis=1)
sinpyeong_weather[row_has_NaN]

,mea_ddhr,ta,ws,rn,lp,ap,date,time
1003,2016-05-12 19:00:00,19.0,NaN,0.0,1010.0,1012.6,2016-05-12,1140.0
1193,2016-05-20 17:00:00,25.5,NaN,0.0,1013.7,1016.2,2016-05-20,1020.0
2652,2016-07-20 12:00:00,31.4,1.9,NaN,1009.8,1012.3,2016-07-20,720.0
2653,2016-07-20 13:00:00,31.6,2.4,NaN,1009.2,1011.7,2016-07-20,780.0
2654,2016-07-20 14:00:00,33.5,3.0,NaN,1009.0,1011.4,2016-07-20,840.0
...,...,...,...,...,...,...,...,...
30058,2019-09-08 21:00:00,22.7,0.0,NaN,1007.6,1009.8,2019-09-08,1260.0
30059,2019-09-08 22:00:00,22.6,0.1,NaN,1008.0,1010.2,2019-09-08,1320.0
30060,2019-09-08 23:00:00,22.5,0.0,NaN,1008.2,1010.4,2019-09-08,1380.0
30061,2019-09-09 00:00:00,22.3,0.2,NaN,1008.1,1010.3,2019-09-09,0.0


##### 1. ta

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(sinpyeong_weather, "ta")

date
2016-08-01     1
2016-08-02     1
2016-08-03     1
2016-10-04     1
2016-10-25     1
2017-07-05    14
2017-07-06    16
2017-07-09     1
2017-10-11     1
2017-10-13     1
2018-03-11     3
2018-07-11     1
2018-11-08     1
2018-12-17     1
2019-05-28     2
2019-10-14     1
dtype: int64

In [ ]:
# 추적하는 코드
compareColumn(sinpyeong_weather, dangjin_weather, "ta", "2017-07-05")

,mea_ddhr,left_ta,right_ta
0,2017-07-05 00:00:00,23.7,23.4
1,2017-07-05 01:00:00,23.6,23.1
2,2017-07-05 02:00:00,23.2,23.3
3,2017-07-05 03:00:00,23.5,23.5
4,2017-07-05 04:00:00,23.5,22.7
5,2017-07-05 05:00:00,23.2,22.9
6,2017-07-05 06:00:00,23.6,23.2
7,2017-07-05 07:00:00,24.3,23.8
8,2017-07-05 08:00:00,25.1,24.6
9,2017-07-05 09:00:00,26.3,26.1


In [ ]:
# 결측치가 5개 이상인 것에 대해 대체한다.
sinpyeong_weather = fillMissingValue(sinpyeong_weather, dangjin_weather, "ta", 5)

In [ ]:
# 추적하는 코드
compareColumn(sinpyeong_weather, dangjin_weather, "ta", "2017-07-05")

,mea_ddhr,left_ta,right_ta
0,2017-07-05 00:00:00,23.7,23.4
1,2017-07-05 01:00:00,23.6,23.1
2,2017-07-05 02:00:00,23.2,23.3
3,2017-07-05 03:00:00,23.5,23.5
4,2017-07-05 04:00:00,23.5,22.7
5,2017-07-05 05:00:00,23.2,22.9
6,2017-07-05 06:00:00,23.6,23.2
7,2017-07-05 07:00:00,24.3,23.8
8,2017-07-05 08:00:00,25.1,24.6
9,2017-07-05 09:00:00,26.3,26.1


In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(sinpyeong_weather, "ta")

date
2016-08-01    1
2016-08-02    1
2016-08-03    1
2016-10-04    1
2016-10-25    1
2017-07-09    1
2017-10-11    1
2017-10-13    1
2018-03-11    3
2018-07-11    1
2018-11-08    1
2018-12-17    1
2019-05-28    2
2019-10-14    1
dtype: int64

##### 2. ws

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(sinpyeong_weather, "ws")

date
2016-05-12    1
2016-05-20    1
2016-07-29    1
2016-08-01    1
2016-08-02    1
2016-08-03    1
2016-10-04    2
2016-10-25    1
2017-01-05    2
2017-07-09    3
2017-09-20    1
2017-10-11    2
2017-10-13    1
2018-03-11    3
2018-06-09    1
2018-07-11    1
2018-07-26    1
2018-11-08    1
2018-12-17    2
2019-02-18    1
2019-05-21    1
2019-05-28    2
2019-08-09    1
2019-10-14    1
dtype: int64

In [ ]:
# 추적하는 코드
compareColumn(sinpyeong_weather, dangjin_weather, "ws", "2018-12-17")

,mea_ddhr,left_ws,right_ws
0,2018-12-17 00:00:00,1.3,2.4
1,2018-12-17 01:00:00,1.1,2.2
2,2018-12-17 02:00:00,NaN,NaN
3,2018-12-17 20:00:00,NaN,NaN
4,2018-12-17 21:00:00,1.6,0.3
5,2018-12-17 22:00:00,1.3,0.6
6,2018-12-17 23:00:00,1.0,0.5


In [ ]:
# 결측치가 5개 이상인 것에 대해 대체한다.
sinpyeong_weather = fillMissingValue(sinpyeong_weather, dangjin_weather, "ws", 5)

In [ ]:
# 추적하는 코드
compareColumn(sinpyeong_weather, dangjin_weather, "ws", "2018-12-17")

,mea_ddhr,left_ws,right_ws
0,2018-12-17 00:00:00,1.3,2.4
1,2018-12-17 01:00:00,1.1,2.2
2,2018-12-17 02:00:00,NaN,NaN
3,2018-12-17 20:00:00,NaN,NaN
4,2018-12-17 21:00:00,1.6,0.3
5,2018-12-17 22:00:00,1.3,0.6
6,2018-12-17 23:00:00,1.0,0.5


In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(sinpyeong_weather, "ws")

date
2016-05-12    1
2016-05-20    1
2016-07-29    1
2016-08-01    1
2016-08-02    1
2016-08-03    1
2016-10-04    2
2016-10-25    1
2017-01-05    2
2017-07-09    3
2017-09-20    1
2017-10-11    2
2017-10-13    1
2018-03-11    3
2018-06-09    1
2018-07-11    1
2018-07-26    1
2018-11-08    1
2018-12-17    2
2019-02-18    1
2019-05-21    1
2019-05-28    2
2019-08-09    1
2019-10-14    1
dtype: int64

##### 3. rn

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(sinpyeong_weather, "rn")

date
2016-07-20    12
2016-07-21     1
2016-08-04     1
2016-10-04     1
2017-05-24    12
2017-05-25     1
2017-08-01    10
2017-08-02     1
2018-03-11     1
2018-04-11    14
2018-04-12     1
2018-05-02     4
2018-05-03     1
2018-12-17     1
2019-03-21    10
2019-03-22     1
2019-07-29    14
2019-07-30     1
2019-08-18    19
2019-08-19     1
2019-09-08    21
2019-09-09     1
dtype: int64

In [ ]:
# 추적하는 코드
compareColumn(sinpyeong_weather, dangjin_weather, "rn", "2019-07-29")

,mea_ddhr,left_rn,right_rn
0,2019-07-29 00:00:00,0.0,0.0
1,2019-07-29 01:00:00,0.0,0.0
2,2019-07-29 02:00:00,0.0,0.0
3,2019-07-29 03:00:00,0.0,0.0
4,2019-07-29 04:00:00,0.0,0.0
5,2019-07-29 05:00:00,0.0,0.0
6,2019-07-29 06:00:00,0.0,0.0
7,2019-07-29 07:00:00,0.0,0.0
8,2019-07-29 08:00:00,0.0,0.0
9,2019-07-29 09:00:00,0.0,0.0


In [ ]:
# 결측치가 5개 이상인 것에 대해 대체한다.
sinpyeong_weather = fillMissingValue(sinpyeong_weather, dangjin_weather, "rn", 5)

In [ ]:
# 추적하는 코드
compareColumn(sinpyeong_weather, dangjin_weather, "rn", "2019-07-29")

,mea_ddhr,left_rn,right_rn
0,2019-07-29 00:00:00,0.0,0.0
1,2019-07-29 01:00:00,0.0,0.0
2,2019-07-29 02:00:00,0.0,0.0
3,2019-07-29 03:00:00,0.0,0.0
4,2019-07-29 04:00:00,0.0,0.0
5,2019-07-29 05:00:00,0.0,0.0
6,2019-07-29 06:00:00,0.0,0.0
7,2019-07-29 07:00:00,0.0,0.0
8,2019-07-29 08:00:00,0.0,0.0
9,2019-07-29 09:00:00,0.0,0.0


In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(sinpyeong_weather, "rn")

date
2016-07-21     1
2016-08-04     1
2016-10-04     1
2017-05-25     1
2017-08-01    10
2017-08-02     1
2018-03-11     1
2018-04-12     1
2018-05-02     4
2018-05-03     1
2018-12-17     1
2019-03-22     1
2019-07-30     1
2019-08-18    19
2019-08-19     1
2019-09-09     1
dtype: int64

##### 4. ap

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(sinpyeong_weather, "ap")

date
2016-08-01    1
2016-08-02    1
2016-08-03    1
2016-10-04    1
2016-10-25    1
2017-07-09    1
2017-10-11    1
2017-10-13    1
2018-03-11    3
2018-11-08    1
2018-12-17    1
2019-05-28    2
2019-10-14    1
dtype: int64

In [ ]:
# 추적하는 코드
compareColumn(sinpyeong_weather, seosan_weather, "ap", "2018-03-11")

-1

In [ ]:
# 결측치가 5개 이상인 것에 대해 대체한다.
sinpyeong_weather = fillMissingValue(sinpyeong_weather, seosan_weather, "ap", 5)

In [ ]:
# 추적하는 코드
compareColumn(sinpyeong_weather, seosan_weather, "ap", "2018-03-11")

-1

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(sinpyeong_weather, "ap")

date
2016-08-01    1
2016-08-02    1
2016-08-03    1
2016-10-04    1
2016-10-25    1
2017-07-09    1
2017-10-11    1
2017-10-13    1
2018-03-11    3
2018-11-08    1
2018-12-17    1
2019-05-28    2
2019-10-14    1
dtype: int64

##### 5. lp

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(sinpyeong_weather, "lp")

date
2016-08-01    1
2016-08-02    1
2016-08-03    1
2016-10-04    1
2016-10-25    1
2017-07-09    1
2017-10-11    1
2017-10-13    1
2018-03-11    3
2018-11-08    1
2018-12-17    1
2019-05-28    2
2019-10-14    1
dtype: int64

In [ ]:
# 추적하는 코드
compareColumn(sinpyeong_weather, seosan_weather, "lp", "2018-03-11")

-1

In [ ]:
# 결측치가 5개 이상인 것에 대해 대체한다.
sinpyeong_weather = fillMissingValue(sinpyeong_weather, seosan_weather, "lp", 5)

In [ ]:
# 추적하는 코드
compareColumn(sinpyeong_weather, seosan_weather, "lp", "2016-08-11")

,mea_ddhr,left_lp,right_lp
0,2016-08-11 00:00:00,1008.1,1007.5
1,2016-08-11 01:00:00,1007.9,1007.4
2,2016-08-11 02:00:00,1007.8,1007.3
3,2016-08-11 03:00:00,1007.6,1007.1
4,2016-08-11 04:00:00,1007.8,1007.3
5,2016-08-11 05:00:00,1008.1,1007.5
6,2016-08-11 06:00:00,1008.6,1008.2
7,2016-08-11 07:00:00,1008.9,1008.4
8,2016-08-11 08:00:00,1009.3,1008.7
9,2016-08-11 09:00:00,1009.3,1008.7


In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(sinpyeong_weather, "lp")

date
2016-08-01    1
2016-08-02    1
2016-08-03    1
2016-10-04    1
2016-10-25    1
2017-07-09    1
2017-10-11    1
2017-10-13    1
2018-03-11    3
2018-11-08    1
2018-12-17    1
2019-05-28    2
2019-10-14    1
dtype: int64

##### 제거 후 확인

In [ ]:
sinpyeong_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34980 entries, 0 to 34979
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   mea_ddhr  34980 non-null  datetime64[ns]
 1   ta        34963 non-null  float64       
 2   ws        34947 non-null  float64       
 3   rn        34934 non-null  float64       
 4   lp        34964 non-null  float64       
 5   ap        34964 non-null  float64       
 6   date      34980 non-null  object        
 7   time      34980 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 2.1+ MB


#### 3. 서산 

##### 1. ta

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(seosan_weather, "ta")

date
2016-07-29    1
2020-01-04    1
dtype: int64

##### 2. hm

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(seosan_weather, "hm")

date
2016-07-29    3
2020-01-04    1
dtype: int64

##### 3. ws

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(seosan_weather, "ws")

date
2016-07-29    3
2017-08-25    1
2019-06-01    1
2020-01-04    1
dtype: int64

##### 4. rn

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(seosan_weather, "rn")

date
2016-04-01    24
2016-04-02    24
2016-04-03    24
2016-04-04    24
2016-04-05    24
              ..
2020-03-27    20
2020-03-28    24
2020-03-29    24
2020-03-30    24
2020-03-31    24
Length: 1459, dtype: int64

In [ ]:
# 추적하는 코드
compareColumn(seosan_weather, dangjin_weather, "rn", "2020-03-27")

,mea_ddhr,left_rn,right_rn
0,2020-03-27 00:00:00,2.3,2.5
1,2020-03-27 01:00:00,NaN,3.0
2,2020-03-27 02:00:00,NaN,1.5
3,2020-03-27 03:00:00,9.5,1.0
4,2020-03-27 04:00:00,NaN,1.0
5,2020-03-27 05:00:00,NaN,0.5
6,2020-03-27 06:00:00,1.7,0.0
7,2020-03-27 07:00:00,NaN,0.0
8,2020-03-27 08:00:00,NaN,0.0
9,2020-03-27 09:00:00,0.0,0.0


In [ ]:
# 결측치가 5개 이상인 것에 대해 대체한다.
seosan_weather = fillMissingValue(seosan_weather, dangjin_weather, "rn", 5)

In [ ]:
# 추적하는 코드
compareColumn(seosan_weather, dangjin_weather, "rn", "2020-03-27")

,mea_ddhr,left_rn,right_rn
0,2020-03-27 00:00:00,2.3,2.5
1,2020-03-27 01:00:00,3.0,3.0
2,2020-03-27 02:00:00,1.5,1.5
3,2020-03-27 03:00:00,9.5,1.0
4,2020-03-27 04:00:00,1.0,1.0
5,2020-03-27 05:00:00,0.5,0.5
6,2020-03-27 06:00:00,1.7,0.0
7,2020-03-27 07:00:00,0.0,0.0
8,2020-03-27 08:00:00,0.0,0.0
9,2020-03-27 09:00:00,0.0,0.0


In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(seosan_weather, "rn")

date
2016-05-27    24
2016-07-16     1
2016-07-29    15
2016-07-30    15
2016-08-12    24
2016-09-08    24
2016-10-06    24
2016-10-17    20
2017-04-20     1
2017-07-09    15
2017-07-24    11
2017-07-31     7
2017-08-01    13
2017-08-02    22
2017-08-07     1
2017-08-20     4
2017-10-11    12
2017-12-18    16
2017-12-26    19
2017-12-28    18
2018-01-02    19
2018-01-03    16
2018-01-04    22
2018-02-08    21
2018-04-05     4
2018-04-16     1
2018-05-12     1
2018-06-21    24
2018-06-26     2
2018-07-01     3
2018-07-24    18
2018-09-08     1
2018-12-17    22
2019-01-31    21
2019-03-14    20
2019-03-15    20
2019-05-31    21
2019-06-01    16
2019-07-25    17
2019-08-02    16
2019-08-03    22
2019-08-12     3
2019-08-17    20
2019-08-18    24
2019-08-19    24
2019-08-20    24
2019-08-30     1
2019-10-01    23
2019-10-02     3
2019-11-17    19
2019-11-18    18
2020-01-01    23
dtype: int64

In [ ]:
# nan 이면 0.0을 넣어준다.
seosan_weather["rn"] = seosan_weather["rn"].apply(lambda x: 0.0 if x != x else x)

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(seosan_weather, "rn")

Series([], dtype: int64)

##### 5. dew

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(seosan_weather, "ws")

date
2016-07-29    3
2017-08-25    1
2019-06-01    1
2020-01-04    1
dtype: int64

##### 6. sp

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(seosan_weather, "sp")

date
2016-07-29    3
2020-01-04    1
dtype: int64

##### 7. lp

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(seosan_weather, "lp")

date
2016-07-29    3
2020-01-04    1
dtype: int64

##### 8. ap

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(seosan_weather, "ap")

date
2016-07-29    3
2020-01-04    1
dtype: int64

##### 9. sr

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(seosan_weather, "sr")

date
2016-04-01    11
2016-04-02    11
2016-04-03    11
2016-04-04    11
2016-04-05    11
              ..
2020-03-27    11
2020-03-28    11
2020-03-29    11
2020-03-30    11
2020-03-31    11
Length: 1461, dtype: int64

In [ ]:
# 일조량이 없을 때는 nan이므로 0.0을 박는다.
seosan_weather["sr"] = seosan_weather["sr"].apply(lambda x: 0.0 if x != x else x)

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(seosan_weather, "sr")

Series([], dtype: int64)

##### 10. ss

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(seosan_weather, "ss")

date
2016-04-01    11
2016-04-02    11
2016-04-03    11
2016-04-04    11
2016-04-05    11
              ..
2020-03-27    11
2020-03-28    11
2020-03-29    11
2020-03-30    11
2020-03-31    11
Length: 1461, dtype: int64

In [ ]:
# 일조량이 없을 때는 nan이므로 0.0을 박는다.
seosan_weather["ss"] = seosan_weather["ss"].apply(lambda x: 0.0 if x != x else x)

In [ ]:
# interpolation 함수를 만드는 threshold 를 알아보기 위해, ws가 nan인 경우 날짜에 대해 몇개가 있는지 확인하자.
getNanGroup(seosan_weather, "ss")

Series([], dtype: int64)

##### 제거 후 확인

In [ ]:
seosan_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34993 entries, 0 to 34992
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   mea_ddhr  34993 non-null  datetime64[ns]
 1   ta        34991 non-null  float64       
 2   hm        34989 non-null  float64       
 3   ws        34987 non-null  float64       
 4   rn        34993 non-null  float64       
 5   dew       34989 non-null  float64       
 6   sp        34989 non-null  float64       
 7   lp        34989 non-null  float64       
 8   ap        34989 non-null  float64       
 9   sr        34993 non-null  float64       
 10  ss        34993 non-null  float64       
 11  ground    34910 non-null  float64       
 12  date      34993 non-null  object        
 13  time      34993 non-null  float64       
dtypes: datetime64[ns](1), float64(12), object(1)
memory usage: 3.7+ MB


#### 4. 송산

In [ ]:
# 전체 nan 확인
is_NaN = songsan_weather.isnull()
row_has_NaN = is_NaN.any(axis=1)
songsan_weather[row_has_NaN]

,mea_ddhr,ta,hm,ws,rn,date,time


#### 5. 송악

In [ ]:
# 전체 nan 확인
is_NaN = songak_weather.isnull()
row_has_NaN = is_NaN.any(axis=1)
songak_weather[row_has_NaN]

,mea_ddhr,ta,hm,ws,rn,date,time


### 4. 파일 저장하기

In [ ]:
# 파일 변환
dangjin_weather.to_csv(os.path.join(DATA_CONV_PATH, "dangjin_weather.csv"), header=True, index=False)

In [ ]:
# 파일 변환
sinpyeong_weather.to_csv(os.path.join(DATA_CONV_PATH, "sinpyeong_weather.csv"), header=True, index=False)

In [ ]:
# 파일 변환
seosan_weather.to_csv(os.path.join(DATA_CONV_PATH, "seosan_weather.csv"), header=True, index=False)

In [ ]:
# 파일 변환
songsan_weather_utc.to_csv(os.path.join(DATA_CONV_PATH, "songsan_weather_utc.csv"), header=True, index=False)
songsan_weather.to_csv(os.path.join(DATA_CONV_PATH, "songsan_weather.csv"), header=True, index=False)
#songsan_weather_utc_INT.to_csv(os.path.join(DATA_CONV_PATH, "songsan_weather_utc_INT.csv"), header=True, index=False)
#songsan_weather_INT.to_csv(os.path.join(DATA_CONV_PATH, "songsan_weather_INT.csv"), header=True, index=False)

In [ ]:
# 파일 변환
songak_weather_utc.to_csv(os.path.join(DATA_CONV_PATH, "songak_weather_utc.csv"), header=True, index=False)
songak_weather.to_csv(os.path.join(DATA_CONV_PATH, "songak_weather.csv"), header=True, index=False)
#songak_weather_utc_INT.to_csv(os.path.join(DATA_CONV_PATH, "songak_weather_utc_INT.csv"), header=True, index=False)
#songak_weather_INT.to_csv(os.path.join(DATA_CONV_PATH, "songak_weather_INT.csv"), header=True, index=False)

### 5. 보간 함수 dictionary 만들기

#### 1. 사용하는 함수


<설명서>
* ret_dict : tuple
    * (where, target, date) -> interpolation function 

=== input ===
1. where  : string        ->   관측소의 위치 (dangjin, ...)
2. target : list          ->   interpolation 하고 싶은 column
3. date   : datetime.date ->   원하는 날짜


=== example === 
dict("dangjin", ["rn"], datetime(~~~))

In [ ]:
import datetime
def interpolation_fn(df, where, target, k=1):
  """
  describe : 기상 데이터를 가지고 시간대 별로 존재하는 기상 특성치를 이은 보간 함수를 만든다.
  <input>
  df : pd.DataFrame -> 날씨 df
  where : string -> 관측소의 위치 (dangjin, ...)
  target : list -> interpolation 하고 싶은 column
  k : int -> 몇차 함수로 보간하는지
  <output>
  ret_dict : tuple -> (where, target, date): interpolation function object
  """
  ret_dict = dict()

  for col in target:
    temp_df = df[["mea_ddhr", "date", "time", col]]
    temp_df = temp_df.dropna()
    date_start = min(df['date'])
    date_end = max(df['date'])
    date = date_start

    while date <= date_end:
      df_date = temp_df[temp_df['date'] == date]
      if len(df_date) < 15:
        print(where, col, date, len(df_date), " : interpolation하기 위한 개수가 부족합니다.")
        date += datetime.timedelta(days=1)
        continue

      # 해당 col에 대한 ipo function 만들기
      x = df_date['time']
      y = df_date[col]
      ipo = spi.splrep(x, y, k=k)
      ret_dict[(where, col, date)] = ipo
      date += datetime.timedelta(days=1)

    # 없는 날짜(데이터에서 없었음)에 대해서 가장 가까운 날짜의 ipo함수를 넣어주는 작업
    keys = ret_dict.keys()
    
    date = date_start
    while date <= date_end:
      key = (where, col, date)
      if key in keys:
        date += datetime.timedelta(days=1)
        continue
      
      delta = 1
      while True:
        if delta > 0:
          tmp_date = date + datetime.timedelta(days=delta)
          tmp_key = (where, col, tmp_date)
          if tmp_key in keys:
            ipo = ret_dict[tmp_key]
            ret_dict[key] = copy.deepcopy(ipo)
            #print(key)
            break
          delta = -delta
        else:
          tmp_date = date - datetime.timedelta(days=-delta)
          tmp_key = (where, col, tmp_date)
          if tmp_key in keys:
            ipo = ret_dict[tmp_key]
            ret_dict[key] = copy.deepcopy(ipo)
            #print(key,tmp_key)
            break
          delta = -delta + 1
      date += datetime.timedelta(days=1)

  return ret_dict

#### 2. 데이터 불러오기

In [ ]:
# dangjin_weather = pd.read_csv(os.path.join(DATA_CONV_PATH, "dangjin_weather.csv"), 
#                       parse_dates=['mea_ddhr', "date"], 
#                       infer_datetime_format=True,encoding = 'ISO-8859-1')
# sinpyeong_weather = pd.read_csv(os.path.join(DATA_CONV_PATH, "sinpyeong_weather.csv"), 
#                       parse_dates=['mea_ddhr', "date"], 
#                       infer_datetime_format=True,encoding = 'ISO-8859-1')
# seosan_weather = pd.read_csv(os.path.join(DATA_CONV_PATH, "seosan_weather.csv"), 
#                       parse_dates=['mea_ddhr', "date"], 
#                       infer_datetime_format=True,encoding = 'ISO-8859-1')
# songsan_weather = pd.read_csv(os.path.join(DATA_CONV_PATH, "songsan_weather.csv"), 
#                       parse_dates=['mea_ddhr', "date"], 
#                       infer_datetime_format=True,encoding = 'ISO-8859-1')
# songak_weather = pd.read_csv(os.path.join(DATA_CONV_PATH, "songak_weather.csv"), 
#                       parse_dates=['mea_ddhr', "date"], 
#                       infer_datetime_format=True,encoding = 'ISO-8859-1')

In [ ]:
dangjin_weather = pd.read_csv(os.path.join(DATA_CONV_PATH, "dangjin_weather.csv"))
sinpyeong_weather = pd.read_csv(os.path.join(DATA_CONV_PATH, "sinpyeong_weather.csv"))
seosan_weather = pd.read_csv(os.path.join(DATA_CONV_PATH, "seosan_weather.csv"))
songsan_weather = pd.read_csv(os.path.join(DATA_CONV_PATH, "songsan_weather.csv"))
songak_weather = pd.read_csv(os.path.join(DATA_CONV_PATH, "songak_weather.csv"))

In [ ]:
dangjin_weather["mea_ddhr"] = dangjin_weather["mea_ddhr"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
sinpyeong_weather["mea_ddhr"] = sinpyeong_weather["mea_ddhr"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
seosan_weather["mea_ddhr"] = seosan_weather["mea_ddhr"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
songsan_weather["mea_ddhr"] = songsan_weather["mea_ddhr"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
songak_weather["mea_ddhr"] = songak_weather["mea_ddhr"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

dangjin_weather["date"] = dangjin_weather["date"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").date())
sinpyeong_weather["date"] = sinpyeong_weather["date"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").date())
seosan_weather["date"] = seosan_weather["date"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").date())
songsan_weather["date"] = songsan_weather["date"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").date())
songak_weather["date"] = songak_weather["date"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").date())

In [ ]:
# ta	ws	rn	lp	ap 에 대한 보간 함수를 만들 수 있음
sinpyeong_weather

,mea_ddhr,ta,ws,rn,lp,ap,date,time
0,2016-04-01 00:00:00,7.2,0.9,0.0,1013.0,1015.7,2016-04-01,0.0
1,2016-04-01 01:00:00,5.6,1.1,0.0,1012.6,1015.3,2016-04-01,60.0
2,2016-04-01 02:00:00,5.1,0.0,0.0,1012.2,1014.9,2016-04-01,120.0
3,2016-04-01 03:00:00,4.7,0.3,0.0,1012.2,1014.9,2016-04-01,180.0
4,2016-04-01 04:00:00,3.7,1.1,0.0,1012.4,1015.1,2016-04-01,240.0
...,...,...,...,...,...,...,...,...
34975,2020-03-31 19:00:00,10.4,2.0,0.0,1012.9,1015.3,2020-03-31,1140.0
34976,2020-03-31 20:00:00,9.4,1.7,0.0,1013.4,1015.8,2020-03-31,1200.0
34977,2020-03-31 21:00:00,8.7,2.4,0.0,1013.9,1016.3,2020-03-31,1260.0
34978,2020-03-31 22:00:00,8.2,2.0,0.0,1013.9,1016.3,2020-03-31,1320.0


In [ ]:
# ta	hm	ws	rn	dew	sp	lp	ap	sr	ss	ground 에 대한 보간 함수를 만들 수 있음
seosan_weather

,mea_ddhr,ta,hm,ws,rn,dew,sp,lp,ap,sr,ss,ground,date,time
0,2016-04-01 00:00:00,8.8,41.0,1.5,0.0,-3.7,4.6,1012.1,1015.6,0.0,0.00,4.0,2016-04-01,0.0
1,2016-04-01 01:00:00,9.2,34.0,1.2,0.0,-5.8,4.0,1011.7,1015.2,0.0,0.00,3.4,2016-04-01,60.0
2,2016-04-01 02:00:00,9.2,33.0,1.1,0.0,-6.2,3.8,1011.3,1014.8,0.0,0.00,3.3,2016-04-01,120.0
3,2016-04-01 03:00:00,7.5,40.0,0.4,0.0,-5.2,4.2,1011.4,1014.9,0.0,0.00,2.8,2016-04-01,180.0
4,2016-04-01 04:00:00,4.8,56.0,0.6,0.0,-3.2,4.8,1011.6,1015.2,0.0,0.00,1.8,2016-04-01,240.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34988,2020-03-31 19:00:00,9.6,46.0,1.1,0.0,-1.4,5.5,1012.3,1015.4,0.0,0.08,10.3,2020-03-31,1140.0
34989,2020-03-31 20:00:00,8.1,53.0,0.2,0.0,-0.9,5.7,1012.9,1016.0,0.0,0.00,8.9,2020-03-31,1200.0
34990,2020-03-31 21:00:00,7.4,59.0,0.6,0.0,-0.1,6.1,1013.4,1016.5,0.0,0.00,8.0,2020-03-31,1260.0
34991,2020-03-31 22:00:00,7.4,58.0,1.6,0.0,-0.3,6.0,1013.3,1016.4,0.0,0.00,7.8,2020-03-31,1320.0


In [ ]:
# ta	hm	ws	rn 에 대한 보간 함수를 만들 수 있음
songsan_weather

,mea_ddhr,ta,hm,ws,rn,date,time
0,2016-03-01 09:00:00,-2.9,63.0,1.0,0.0,2016-03-01,0.0
1,2016-03-01 10:00:00,-1.4,57.0,0.4,0.0,2016-03-01,60.0
2,2016-03-01 11:00:00,-0.2,53.0,0.8,0.0,2016-03-01,120.0
3,2016-03-01 12:00:00,0.6,50.0,1.1,0.0,2016-03-01,180.0
4,2016-03-01 13:00:00,1.6,48.0,1.2,0.0,2016-03-01,240.0
...,...,...,...,...,...,...,...
35803,2020-04-01 04:00:00,8.5,58.0,1.4,0.0,2020-03-31,1140.0
35804,2020-04-01 05:00:00,8.3,56.0,1.1,0.0,2020-03-31,1200.0
35805,2020-04-01 06:00:00,8.4,51.0,1.3,0.0,2020-03-31,1260.0
35806,2020-04-01 07:00:00,9.4,47.0,1.7,0.0,2020-03-31,1320.0


In [ ]:
# ta	hm	ws	rn 에 대한 보간 함수를 만들 수 있음
songak_weather

,mea_ddhr,ta,hm,ws,rn,date,time
0,2016-03-01 09:00:00,-3.4,62.0,1.3,0.0,2016-03-01,0.0
1,2016-03-01 10:00:00,-1.7,57.0,0.5,0.0,2016-03-01,60.0
2,2016-03-01 11:00:00,-0.5,52.0,0.7,0.0,2016-03-01,120.0
3,2016-03-01 12:00:00,0.4,50.0,1.1,0.0,2016-03-01,180.0
4,2016-03-01 13:00:00,1.5,47.0,1.0,0.0,2016-03-01,240.0
...,...,...,...,...,...,...,...
35803,2020-04-01 04:00:00,8.5,58.0,1.4,0.0,2020-03-31,1140.0
35804,2020-04-01 05:00:00,8.3,56.0,1.1,0.0,2020-03-31,1200.0
35805,2020-04-01 06:00:00,8.4,51.0,1.3,0.0,2020-03-31,1260.0
35806,2020-04-01 07:00:00,9.4,47.0,1.7,0.0,2020-03-31,1320.0


#### 3. 사용할 dictionary 제작

In [ ]:
weatherDataList = [dangjin_weather, sinpyeong_weather, seosan_weather, songsan_weather, songak_weather]
locList = ["dangjin", "sinpyeong", "seosan", "songsan", "songak"]
featureList = [
               ["ta",	"hm",	"ws",	"rn"],
               ["ta",	"ws",	"rn",	"lp",	"ap"],
               ["ta",	"hm",	"ws",	"rn",	"dew",	"sp",	"lp",	"ap",	"sr",	"ss",	"ground"],
               ["ta",	"hm",	"ws",	"rn"],
               ["ta",	"hm",	"ws",	"rn"]
]


In [ ]:
# key -> (where, target, date)
ipo_dict = dict()
for i in range(len(weatherDataList)):
    ipo_dict.update(interpolation_fn(weatherDataList[i], locList[i], featureList[i]))

dangjin ta 2016-07-29 11  : interpolation하기 위한 개수가 부족합니다.
dangjin ta 2018-12-17 7  : interpolation하기 위한 개수가 부족합니다.
dangjin ta 2019-08-17 13  : interpolation하기 위한 개수가 부족합니다.
dangjin ta 2019-08-18 0  : interpolation하기 위한 개수가 부족합니다.
dangjin ta 2019-08-19 0  : interpolation하기 위한 개수가 부족합니다.
dangjin ta 2019-08-20 13  : interpolation하기 위한 개수가 부족합니다.
dangjin ta 2019-11-17 11  : interpolation하기 위한 개수가 부족합니다.
dangjin ta 2019-11-18 12  : interpolation하기 위한 개수가 부족합니다.
dangjin hm 2016-07-29 11  : interpolation하기 위한 개수가 부족합니다.
dangjin hm 2018-12-17 7  : interpolation하기 위한 개수가 부족합니다.
dangjin hm 2019-08-17 13  : interpolation하기 위한 개수가 부족합니다.
dangjin hm 2019-08-18 0  : interpolation하기 위한 개수가 부족합니다.
dangjin hm 2019-08-19 0  : interpolation하기 위한 개수가 부족합니다.
dangjin hm 2019-08-20 13  : interpolation하기 위한 개수가 부족합니다.
dangjin hm 2019-11-17 11  : interpolation하기 위한 개수가 부족합니다.
dangjin hm 2019-11-18 12  : interpolation하기 위한 개수가 부족합니다.
dangjin ws 2016-07-29 11  : interpolation하기 위한 개수가 부족합니다.
dangjin ws 2018-12-1

In [ ]:
# dictionary 테스트
from datetime import date, time

where = "dangjin"
col = "ta"
d = date(2018, 6, 21)

key = (where, col, d)
ipo_dict[key]

(array([   0.,    0.,   60.,  120.,  180.,  240.,  300.,  360.,  420.,
         480.,  540.,  600.,  660.,  720.,  780.,  840.,  900.,  960.,
        1020., 1080., 1260., 1320., 1380., 1380.]),
 array([19.3, 18.9, 18.8, 18.9, 19.1, 19. , 18.5, 18.7, 18.6, 20.3, 21.7,
        23.8, 25.3, 25.8, 26.8, 27.5, 27.4, 27.5, 26.3, 21.3, 20.2, 19.4,
         0. ,  0. ]),
 1)

##### 4. Dictionary 저장하기

In [ ]:
import pickle
from sklearn.externals import joblib
joblib.dump(ipo_dict, os.path.join(MODEL_PATH, "ipo_dict.pkl"))

['/content/gdrive/My Drive/weather_bigcontest/4_code/model/ipo_dict.pkl']

In [ ]:
table = np.array([
              ["TARGET : ", "", "", "", "", "", "", "", "", "", ""],
               ["ta",	"hm",	"ws",	"rn", "", "", "","", "", "", ""],
               ["ta",	"ws",	"rn",	"lp",	"ap", "", "","", "", "", ""],
               ["ta",	"hm",	"ws",	"rn",	"dew",	"sp",	"lp",	"ap",	"sr",	"ss",	"ground"],
               ["ta",	"hm",	"ws",	"rn", "", "", "","", "", "", ""],
               ["ta",	"hm",	"ws",	"rn", "", "", "","", "", "", ""],
]).transpose()
newrow = ["DATE : ", "datetime.date(1995-03-16)",  "", "", "", ""]
table = np.vstack([table, newrow])
newrow = ["USAGE : ", "ipo_dict[('dangjin', 'ta', datetime.date(1995-03-16))]",  "", "", "", ""]
table = np.vstack([table, newrow])

In [ ]:
IPO_DICT_READ_ME = pd.DataFrame(data=table, columns=["WHERE : ", "dangjin", "sinpyeong", "seosan", "songsan", "songak"])
IPO_DICT_READ_ME

,WHERE :,dangjin,sinpyeong,seosan,songsan,songak
0,TARGET :,ta,ta,ta,ta,ta
1,,hm,ws,hm,hm,hm
2,,ws,rn,ws,ws,ws
3,,rn,lp,rn,rn,rn
4,,,ap,dew,,
5,,,,sp,,
6,,,,lp,,
7,,,,ap,,
8,,,,sr,,
9,,,,ss,,


In [ ]:
# 파일 변환
IPO_DICT_READ_ME.to_csv(os.path.join(MODEL_PATH, "IPO_DICT_READ_ME.csv"), header=True, index=False)